In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from tqdm.notebook import tqdm
import random
import pandas as pd
from scipy.io import savemat
import scipy
import matplotlib.cm as cm
from utils import *

In [ ]:
# experimental task
def task(freq_seq, lm, hm, s, n_trials = 10, n_tones = 3, p_low = 0.5, p_back = 0.3):
    log_freq_seq_mid = np.median(exptFreqSeqArray)
    log_freq_low = [lm,s]#[log_freq_seq_mid - 0.15,0.1]  #low freq condition is gaussian 
    log_freq_high = [hm,s]#[log_freq_seq_mid + 0.15,0.1] #high freq condition is gaussian
    trial_tones = []
    dist_chosen = np.array([])
    kind_of_tones = []

    for trial in range(n_trials):
        signal_rand = np.random.random()
        low_dist = signal_rand < p_low #choosing true tone from either low or high condition
        tones = []
        tone_kind = []
        for n_tone in range(n_tones):
            signal_back = np.random.random()
            background = signal_back < p_back #choosing background or true tone
            if background:
                nearest_log_tone = np.random.choice(exptFreqSeqArray)
                #background freq is chosen from a uniform distribution
                tone_kind.append(0)
            else: 
                if low_dist:
                    tone = min(max(np.random.randn()*log_freq_low[1] + log_freq_low[0],\
                                   exptFreqSeqArray[0]),exptFreqSeqArray[-1])                    
                    tone_kind.append(1)
                else:
                    tone = min(max(np.random.randn()*log_freq_high[1] + log_freq_high[0],\
                                   exptFreqSeqArray[0]),exptFreqSeqArray[-1])
                    tone_kind.append(2)
                nearest_log_tone = exptFreqSeqArray[np.argmin(np.abs(exptFreqSeqArray - tone))]
            nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**nearest_log_tone))]        
            tones.append(nearest_tone)
        trial_tones.append(tones)
        dist_chosen = np.append(dist_chosen, 1-low_dist)
        kind_of_tones.append(tone_kind)
    return trial_tones, dist_chosen, kind_of_tones, log_freq_low, log_freq_high

In [ ]:
freqSeq = np.arange(90,3000,1) #array of possible true tones
exptFreqSeqArray = np.arange(np.log10(freqSeq[0]), np.log10(freqSeq[-1]), np.log10(1003/1000)*40)
logFreqBins = exptFreqSeqArray[np.arange(1,len(exptFreqSeqArray),2)]#(exptFreqSeqArray[:-1]+exptFreqSeqArray[1:])/2
expt_freq_seq_mid = np.median(exptFreqSeqArray)
low_dist = [expt_freq_seq_mid - 0.15,0.1]
high_dist = [expt_freq_seq_mid + 0.15,0.1]
    
def trialCategories(trial_tones, trial_behaviour, corrans):
    """
    dividing trials into different categories : 
    1. all trials with tones from gaussian distributions
    2. all trials with one tone as distractor from extremes
    3. all trials with one tone as distractor from gaussian distribution
    4. all trials with two tones as distractor from opp extremes
    5. all trials with two tones as distractors (one from opp and other from same side extreme)
    
    These will be stored in arrays where the first 3 columns are the tones. 4th is the subject's behaviour.
    5th is the correct answer.
    6th and 7th is optimal subject's reponses given the strategy of taking an average of all the tones.
    8th and 9th is optimal subject's responses given the strategy of ignoring the distractor tones and taking 
    an average of the remaining tones.
    10th is optimal subject's responses given the strategy of taking majority category (i.e. 2 out of 3).
    11th and 12th is optimal subject's responses given the strategy of using only the 
    highest tone for classification.
    
    As of 04-27 I think that pback=1 in the model corresponds to taking an average over all the tones 
    (because in that case we think that each of the tones has signal in it). However taking pback=0 in the model 
    corresponds to the other two cases, because when pback = 0 for a particular tone it implies that the tone 
    does not give information about the category
    """
    
    allGaussLow = np.zeros((len(trial_behaviour),13))
    allGaussHigh = np.zeros((len(trial_behaviour),13))
    distGaussHigh = np.zeros((len(trial_behaviour),13))
    distGaussLow = np.zeros((len(trial_behaviour),13))
    distLowOtherLow = np.zeros((len(trial_behaviour),13))
    distLowOtherHigh = np.zeros((len(trial_behaviour),13))
    distHighOtherLow = np.zeros((len(trial_behaviour),13))
    distHighOtherHigh = np.zeros((len(trial_behaviour),13))
    distLowOtherLowHigh = np.zeros((len(trial_behaviour),13))
    distHighOtherLowHigh = np.zeros((len(trial_behaviour),13))
    distTwoExtrOtherGauss = np.zeros((len(trial_behaviour),13))
    distTwoExtrSameGauss = np.zeros((len(trial_behaviour),13))
    distThreeExtr = np.zeros((len(trial_behaviour),13))
    distThreeExtrTwoSame = np.zeros((len(trial_behaviour),13))
    oneDistractor = np.zeros((len(trial_behaviour),5))
    twoDistractors = np.zeros((len(trial_behaviour),5))
    
    n_tones = len(trial_tones[0])
    print(high_dist[0]+2*high_dist[1], 
          low_dist[0]-2*low_dist[1], 
          expt_freq_seq_mid)
    
    iAllGaussLow = 0; iAllGaussHigh = 0; iDistGaussHigh = 0; iDistGaussLow = 0; iDistLowOtherLow = 0; 
    iDistLowOtherHigh = 0; iDistHighOtherLow = 0; iDistHighOtherHigh = 0; iDistLowOtherLowHigh = 0;
    iDistHighOtherLowHigh = 0; iDistTwoExtrOtherGauss = 0; iDistTwoExtrSameGauss = 0; iDistThreeExtr = 0; 
    iDistThreeExtrTwoSame = 0; iOneDistractor = 0; iTwoDistractors = 0;
    
    for i_trial in range(len(trial_tones)):
        if (sum(trial_tones[i_trial]<expt_freq_seq_mid)==3 
            & sum(trial_tones[i_trial]>(low_dist[0]-2*low_dist[1]))==3):
            allGaussLow[iAllGaussLow,:3] = trial_tones[i_trial,:]
            allGaussLow[iAllGaussLow,3] = trial_behaviour[i_trial]
            allGaussLow[iAllGaussLow,4] = corrans[i_trial]
            allGaussLow[iAllGaussLow,5] = np.mean(allGaussLow[iAllGaussLow,:3])
            allGaussLow[iAllGaussLow,6] = allGaussLow[iAllGaussLow,5] > expt_freq_seq_mid 
            allGaussLow[iAllGaussLow,7] = allGaussLow[iAllGaussLow,5]
            allGaussLow[iAllGaussLow,8] = allGaussLow[iAllGaussLow,6]
            allGaussLow[iAllGaussLow,9] = allGaussLow[iAllGaussLow,6]
            allGaussLow[iAllGaussLow,10] = np.max(trial_tones[i_trial,:]) 
            allGaussLow[iAllGaussLow,11] = allGaussLow[iAllGaussLow,10] > expt_freq_seq_mid
            allGaussLow[iAllGaussLow,12] = int(i_trial)
            iAllGaussLow += 1
            
        elif (sum(trial_tones[i_trial]>expt_freq_seq_mid)==3 
              & sum(trial_tones[i_trial]<(high_dist[0]+2*high_dist[1]))==3): 
            allGaussHigh[iAllGaussHigh,:3] = trial_tones[i_trial,:]
            allGaussHigh[iAllGaussHigh,3] = trial_behaviour[i_trial]
            allGaussHigh[iAllGaussHigh,4] = corrans[i_trial]
            allGaussHigh[iAllGaussHigh,5] = np.mean(allGaussHigh[iAllGaussHigh,:3])
            allGaussHigh[iAllGaussHigh,6] = allGaussHigh[iAllGaussHigh,5] > expt_freq_seq_mid 
            allGaussHigh[iAllGaussHigh,7] = allGaussHigh[iAllGaussHigh,5]
            allGaussHigh[iAllGaussHigh,8] = allGaussHigh[iAllGaussHigh,6]
            allGaussHigh[iAllGaussHigh,9] = allGaussHigh[iAllGaussHigh,6]  
            allGaussHigh[iAllGaussHigh,10] = np.max(trial_tones[i_trial,:]) 
            allGaussHigh[iAllGaussHigh,11] = allGaussHigh[iAllGaussHigh,10] > expt_freq_seq_mid
            allGaussHigh[iAllGaussHigh,12] = int(i_trial)
            iAllGaussHigh += 1
            
        elif (sum(trial_tones[i_trial]<expt_freq_seq_mid)==1
              and sum(trial_tones[i_trial]>expt_freq_seq_mid)==2
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==0
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==0):
            distGaussLow[iDistGaussLow,:3] = trial_tones[i_trial,:]
            distGaussLow[iDistGaussLow,3] = trial_behaviour[i_trial]
            distGaussLow[iDistGaussLow,4] = corrans[i_trial]
            distGaussLow[iDistGaussLow,5] = np.mean(distGaussLow[iDistGaussLow,:3])
            distGaussLow[iDistGaussLow,6] = distGaussLow[iDistGaussLow,5] > expt_freq_seq_mid
            distGaussLow[iDistGaussLow,7] = np.mean(trial_tones[i_trial]
                                                         [trial_tones[i_trial]>expt_freq_seq_mid])
            distGaussLow[iDistGaussLow,8] = distGaussLow[iDistGaussLow,7] > expt_freq_seq_mid
            distGaussLow[iDistGaussLow,9] = 1
            distGaussLow[iDistGaussLow,10] = np.max(trial_tones[i_trial,:]) 
            distGaussLow[iDistGaussLow,11] = distGaussLow[iDistGaussLow,10] > expt_freq_seq_mid 
            distGaussLow[iDistGaussLow,12] = int(i_trial)
            #print(distGaussLow[iDistGaussLow,:3])
            iDistGaussLow += 1
            #pdb.set_trace()    
        
        elif (sum(trial_tones[i_trial]<expt_freq_seq_mid)==2
              and sum(trial_tones[i_trial]>expt_freq_seq_mid)==1
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==0
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==0):
            distGaussHigh[iDistGaussHigh,:3] = trial_tones[i_trial,:]
            distGaussHigh[iDistGaussHigh,3] = trial_behaviour[i_trial]
            distGaussHigh[iDistGaussHigh,4] = corrans[i_trial]
            distGaussHigh[iDistGaussHigh,5] = np.mean(distGaussHigh[iDistGaussHigh,:3])
            distGaussHigh[iDistGaussHigh,6] = distGaussHigh[iDistGaussHigh,5] > expt_freq_seq_mid
            distGaussHigh[iDistGaussHigh,7] = np.mean(trial_tones[i_trial]
                                                         [trial_tones[i_trial]<expt_freq_seq_mid])
            distGaussHigh[iDistGaussHigh,8] = distGaussHigh[iDistGaussHigh,7] > expt_freq_seq_mid
            distGaussHigh[iDistGaussHigh,9] = 0
            distGaussHigh[iDistGaussHigh,10] = np.max(trial_tones[i_trial,:]) 
            distGaussHigh[iDistGaussHigh,11] = distGaussHigh[iDistGaussHigh,10] > expt_freq_seq_mid 
            distGaussHigh[iDistGaussHigh,12] = int(i_trial)
            #print(distGaussHigh[iDistGaussHigh,:3])
            iDistGaussHigh += 1
            #pdb.set_trace()    
        
        elif (sum(trial_tones[i_trial]<expt_freq_seq_mid)==3 
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==1):
            distLowOtherLow[iDistLowOtherLow,:3] = trial_tones[i_trial,:]
            distLowOtherLow[iDistLowOtherLow,3] = trial_behaviour[i_trial]
            distLowOtherLow[iDistLowOtherLow,4] = corrans[i_trial]
            distLowOtherLow[iDistLowOtherLow,5] = np.mean(distLowOtherLow[iDistLowOtherLow,:3])
            distLowOtherLow[iDistLowOtherLow,6] = distLowOtherLow[iDistLowOtherLow,5] > expt_freq_seq_mid
            distLowOtherLow[iDistLowOtherLow,7] = np.mean(trial_tones[i_trial]
                                                         [trial_tones[i_trial]>(low_dist[0]-2*low_dist[1])])
            distLowOtherLow[iDistLowOtherLow,8] = distLowOtherLow[iDistLowOtherLow,7] >expt_freq_seq_mid
            distLowOtherLow[iDistLowOtherLow,9] = 0
            distLowOtherLow[iDistLowOtherLow,10] = np.max(trial_tones[i_trial,:]) 
            distLowOtherLow[iDistLowOtherLow,11] = distLowOtherLow[iDistLowOtherLow,10] > expt_freq_seq_mid 
            distLowOtherLow[iDistLowOtherLow,12] = int(i_trial)
            iDistLowOtherLow += 1
            
        elif (sum(trial_tones[i_trial]>expt_freq_seq_mid)==3 
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==1):
            distHighOtherHigh[iDistHighOtherHigh,:3] = trial_tones[i_trial,:]
            distHighOtherHigh[iDistHighOtherHigh,3] = trial_behaviour[i_trial]
            distHighOtherHigh[iDistHighOtherHigh,4] = corrans[i_trial]
            distHighOtherHigh[iDistHighOtherHigh,5] = np.mean(distHighOtherHigh[iDistHighOtherHigh,:3])
            distHighOtherHigh[iDistHighOtherHigh,6] = distHighOtherHigh[iDistHighOtherHigh,5] > expt_freq_seq_mid
            distHighOtherHigh[iDistHighOtherHigh,7] = np.mean(trial_tones[i_trial]
                                                         [trial_tones[i_trial]<(high_dist[0]+2*high_dist[1])])
            distHighOtherHigh[iDistHighOtherHigh,8] = distHighOtherHigh[iDistHighOtherHigh,7] > expt_freq_seq_mid
            distHighOtherHigh[iDistHighOtherHigh,9] = 1
            distHighOtherHigh[iDistHighOtherHigh,10] = np.max(trial_tones[i_trial,:]) 
            distHighOtherHigh[iDistHighOtherHigh,11] = distHighOtherHigh[iDistHighOtherHigh,10] > expt_freq_seq_mid 
            distHighOtherHigh[iDistHighOtherHigh,12] = int(i_trial)
            iDistHighOtherHigh += 1
                
        elif (sum(trial_tones[i_trial]>expt_freq_seq_mid)==2 
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==0
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==1):    
            distLowOtherHigh[iDistLowOtherHigh,:3] = trial_tones[i_trial,:]
            distLowOtherHigh[iDistLowOtherHigh,3] = trial_behaviour[i_trial]
            distLowOtherHigh[iDistLowOtherHigh,4] = corrans[i_trial]
            distLowOtherHigh[iDistLowOtherHigh,5] = np.mean(distLowOtherHigh[iDistLowOtherHigh,:3])
            distLowOtherHigh[iDistLowOtherHigh,6] = distLowOtherHigh[iDistLowOtherHigh,5] > expt_freq_seq_mid
            distLowOtherHigh[iDistLowOtherHigh,7] = np.mean(trial_tones[i_trial]
                                                         [trial_tones[i_trial]>expt_freq_seq_mid])
            distLowOtherHigh[iDistLowOtherHigh,8] = distLowOtherHigh[iDistLowOtherHigh,7] > expt_freq_seq_mid
            distLowOtherHigh[iDistLowOtherHigh,9] = 1
            distLowOtherHigh[iDistLowOtherHigh,10] = np.max(trial_tones[i_trial,:]) 
            distLowOtherHigh[iDistLowOtherHigh,11] = distLowOtherHigh[iDistLowOtherHigh,10] > expt_freq_seq_mid 
            distLowOtherHigh[iDistLowOtherHigh,12] = int(i_trial)
            iDistLowOtherHigh += 1
            
        elif (sum(trial_tones[i_trial]<expt_freq_seq_mid)==2 
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==1
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==0):
            distHighOtherLow[iDistHighOtherLow,:3] = trial_tones[i_trial,:]
            distHighOtherLow[iDistHighOtherLow,3] = trial_behaviour[i_trial]
            distHighOtherLow[iDistHighOtherLow,4] = corrans[i_trial]
            distHighOtherLow[iDistHighOtherLow,5] = np.mean(distHighOtherLow[iDistHighOtherLow,:3])
            distHighOtherLow[iDistHighOtherLow,6] = distHighOtherLow[iDistHighOtherLow,5] > expt_freq_seq_mid
            distHighOtherLow[iDistHighOtherLow,7] = np.mean(trial_tones[i_trial]
                                                         [trial_tones[i_trial]<expt_freq_seq_mid])
            distHighOtherLow[iDistHighOtherLow,8] = distHighOtherLow[iDistHighOtherLow,7] > expt_freq_seq_mid
            distHighOtherLow[iDistHighOtherLow,9] = 0
            distHighOtherLow[iDistHighOtherLow,10] = np.max(trial_tones[i_trial,:]) 
            distHighOtherLow[iDistHighOtherLow,11] = distHighOtherLow[iDistHighOtherLow,10] > expt_freq_seq_mid 
            distHighOtherLow[iDistHighOtherLow,12] = int(i_trial)
            iDistHighOtherLow += 1
            
        elif (sum(trial_tones[i_trial]>expt_freq_seq_mid)==1 
              and sum(trial_tones[i_trial]<expt_freq_seq_mid)==2 
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==0
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==1):    
            distLowOtherLowHigh[iDistLowOtherLowHigh,:3] = trial_tones[i_trial,:]
            distLowOtherLowHigh[iDistLowOtherLowHigh,3] = trial_behaviour[i_trial]
            distLowOtherLowHigh[iDistLowOtherLowHigh,4] = corrans[i_trial]
            distLowOtherLowHigh[iDistLowOtherLowHigh,5] = np.mean(distLowOtherLowHigh[iDistLowOtherLowHigh,:3])
            distLowOtherLowHigh[iDistLowOtherLowHigh,6] = distLowOtherLowHigh[iDistLowOtherLowHigh,5] > expt_freq_seq_mid
            distLowOtherLowHigh[iDistLowOtherLowHigh,7] = None
            distLowOtherLowHigh[iDistLowOtherLowHigh,8] = None
            distLowOtherLowHigh[iDistLowOtherLowHigh,9] = 0
            distLowOtherLowHigh[iDistLowOtherLowHigh,10] = np.max(trial_tones[i_trial,:]) 
            distLowOtherLowHigh[iDistLowOtherLowHigh,11] = distLowOtherLowHigh[iDistLowOtherLowHigh,10] > expt_freq_seq_mid 
            distLowOtherLowHigh[iDistLowOtherLowHigh,12] = int(i_trial)
            #print('a',distLowOtherLowHigh[iDistLowOtherLowHigh,:3])
            iDistLowOtherLowHigh += 1    
            
        elif (sum(trial_tones[i_trial]>expt_freq_seq_mid)==2 
              and sum(trial_tones[i_trial]<expt_freq_seq_mid)==1 
              and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==1
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==0):    
            distHighOtherLowHigh[iDistHighOtherLowHigh,:3] = trial_tones[i_trial,:]
            distHighOtherLowHigh[iDistHighOtherLowHigh,3] = trial_behaviour[i_trial]
            distHighOtherLowHigh[iDistHighOtherLowHigh,4] = corrans[i_trial]
            distHighOtherLowHigh[iDistHighOtherLowHigh,5] = np.mean(distHighOtherLowHigh[iDistHighOtherLowHigh,:3])
            distHighOtherLowHigh[iDistHighOtherLowHigh,6] = distHighOtherLowHigh[iDistHighOtherLowHigh,5] > expt_freq_seq_mid
            distHighOtherLowHigh[iDistHighOtherLowHigh,7] = None
            distHighOtherLowHigh[iDistHighOtherLowHigh,8] = None
            distHighOtherLowHigh[iDistHighOtherLowHigh,9] = 1
            distHighOtherLowHigh[iDistHighOtherLowHigh,10] = np.max(trial_tones[i_trial,:]) 
            distHighOtherLowHigh[iDistHighOtherLowHigh,11] = distHighOtherLowHigh[iDistHighOtherLowHigh,10] > expt_freq_seq_mid 
            distHighOtherLowHigh[iDistHighOtherLowHigh,12] = int(i_trial)
            #print('a',distHighOtherLowHigh[iDistHighOtherLowHigh,:3])
            iDistHighOtherLowHigh += 1 
            
        elif (sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==2 
              and sum(trial_tones[i_trial]<expt_freq_seq_mid)==1
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==0):
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,:3] = trial_tones[i_trial,:]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,3] = trial_behaviour[i_trial]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,4] = corrans[i_trial]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,5] = np.mean(distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,:3])
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,6] = distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,5] > expt_freq_seq_mid
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,7] = trial_tones[i_trial][np.argmin(abs(trial_tones[i_trial]-expt_freq_seq_mid))]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,8] = distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,7] > expt_freq_seq_mid
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,10] = np.max(trial_tones[i_trial,:]) 
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,11] = distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,10] > expt_freq_seq_mid 
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,12] = int(i_trial)
            #print(distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,:3])
            iDistTwoExtrOtherGauss += 1
              
        elif (sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==0 
              and sum(trial_tones[i_trial]>expt_freq_seq_mid)==1
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==2):            
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,:3] = trial_tones[i_trial,:]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,3] = trial_behaviour[i_trial]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,4] = corrans[i_trial]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,5] = np.mean(distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,:3])
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,6] = distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,5] > expt_freq_seq_mid
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,7] = trial_tones[i_trial][np.argmin(abs(trial_tones[i_trial]-expt_freq_seq_mid))]
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,8] = distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,7] > expt_freq_seq_mid
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,10] = np.max(trial_tones[i_trial,:]) 
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,11] = distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,10] > expt_freq_seq_mid 
            distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,12] = int(i_trial)
            #print(distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,:3])
            iDistTwoExtrOtherGauss += 1  
            
        elif (sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==0 
              and sum(trial_tones[i_trial]<expt_freq_seq_mid)==3
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==2):
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,:3] = trial_tones[i_trial,:]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,3] = trial_behaviour[i_trial]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,4] = corrans[i_trial]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,5] = np.mean(distTwoExtrSameGauss[iDistTwoExtrSameGauss,:3])
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,6] = distTwoExtrSameGauss[iDistTwoExtrSameGauss,5] > expt_freq_seq_mid
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,7] = trial_tones[i_trial][np.argmin(abs(trial_tones[i_trial]-expt_freq_seq_mid))]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,8] = distTwoExtrSameGauss[iDistTwoExtrSameGauss,7] > expt_freq_seq_mid
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,10] = np.max(trial_tones[i_trial,:]) 
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,11] = distTwoExtrSameGauss[iDistTwoExtrSameGauss,10] > expt_freq_seq_mid 
            #print(distTwoExtrSameGauss[iDistTwoExtrSameGauss,[0,1,2,3,4,6,8,9]])
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,12] = int(i_trial)
            iDistTwoExtrSameGauss += 1
              
        elif (sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==2
              and sum(trial_tones[i_trial]>expt_freq_seq_mid)==3
              and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==0):            
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,:3] = trial_tones[i_trial,:]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,3] = trial_behaviour[i_trial]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,4] = corrans[i_trial]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,5] = np.mean(distTwoExtrSameGauss[iDistTwoExtrSameGauss,:3])
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,6] = distTwoExtrSameGauss[iDistTwoExtrSameGauss,5] > expt_freq_seq_mid
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,7] = trial_tones[i_trial][np.argmin(abs(trial_tones[i_trial]-expt_freq_seq_mid))]
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,8] = distTwoExtrSameGauss[iDistTwoExtrSameGauss,7] > expt_freq_seq_mid
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,10] = np.max(trial_tones[i_trial,:]) 
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,11] = distTwoExtrSameGauss[iDistTwoExtrSameGauss,10] > expt_freq_seq_mid 
            #print(distTwoExtrSameGauss[iDistTwoExtrSameGauss,[0,1,2,3,4,6,8,9]])
            distTwoExtrSameGauss[iDistTwoExtrSameGauss,12] = int(i_trial)
            iDistTwoExtrSameGauss += 1    
            
        elif (sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==3):
            distThreeExtr[iDistThreeExtr,:3] = trial_tones[i_trial,:]
            distThreeExtr[iDistThreeExtr,3] = trial_behaviour[i_trial]
            distThreeExtr[iDistThreeExtr,4] = corrans[i_trial]
            distThreeExtr[iDistThreeExtr,5] = np.mean(distThreeExtr[iDistThreeExtr,:3])
            distThreeExtr[iDistThreeExtr,6] = distThreeExtr[iDistThreeExtr,5] > expt_freq_seq_mid
            distThreeExtr[iDistThreeExtr,7] = None
            distThreeExtr[iDistThreeExtr,8] = None
            distThreeExtr[iDistThreeExtr,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distThreeExtr[iDistThreeExtr,10] = np.max(trial_tones[i_trial,:]) 
            distThreeExtr[iDistThreeExtr,11] = distThreeExtr[iDistThreeExtr,10] > expt_freq_seq_mid 
            distThreeExtr[iDistThreeExtr,12] = int(i_trial)
            #print(distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,[0,1,2,3,5,6,7,8,9]])
            iDistThreeExtr += 1
              
        elif (sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==3):
            distThreeExtr[iDistThreeExtr,:3] = trial_tones[i_trial,:]
            distThreeExtr[iDistThreeExtr,3] = trial_behaviour[i_trial]
            distThreeExtr[iDistThreeExtr,4] = corrans[i_trial]
            distThreeExtr[iDistThreeExtr,5] = np.mean(distThreeExtr[iDistThreeExtr,:3])
            distThreeExtr[iDistThreeExtr,6] = distThreeExtr[iDistThreeExtr,5] > expt_freq_seq_mid
            distThreeExtr[iDistThreeExtr,7] = None
            distThreeExtr[iDistThreeExtr,8] = None
            distThreeExtr[iDistThreeExtr,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distThreeExtr[iDistThreeExtr,10] = np.max(trial_tones[i_trial,:]) 
            distThreeExtr[iDistThreeExtr,11] = distThreeExtr[iDistThreeExtr,10] > expt_freq_seq_mid 
            distThreeExtr[iDistThreeExtr,12] = int(i_trial)
            #print(distTwoExtrOtherGauss[iDistTwoExtrOtherGauss,[0,1,2,3,5,6,7,8,9]])
            iDistThreeExtr += 1    
       
        elif (sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==2
             and sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==1):
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,:3] = trial_tones[i_trial,:]
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,3] = trial_behaviour[i_trial]
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,4] = corrans[i_trial]
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,5] = np.mean(distThreeExtrTwoSame[iDistThreeExtrTwoSame,:3])
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,6] = distThreeExtrTwoSame[iDistThreeExtrTwoSame,5] > expt_freq_seq_mid
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,7] = None
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,8] = None
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,10] = np.max(trial_tones[i_trial,:]) 
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,11] = distThreeExtrTwoSame[iDistThreeExtrTwoSame,10] > expt_freq_seq_mid 
            #print(distThreeExtrTwoSame[iDistThreeExtrTwoSame,[0,1,2,3,4,5,6,7,8,9]])
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,12] = int(i_trial)
            iDistThreeExtrTwoSame += 1
              
        elif (sum(trial_tones[i_trial]>(high_dist[0]+2*high_dist[1]))==2
             and sum(trial_tones[i_trial]<(low_dist[0]-2*low_dist[1]))==1):
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,:3] = trial_tones[i_trial,:]
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,3] = trial_behaviour[i_trial]
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,4] = corrans[i_trial]
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,5] = np.mean(distThreeExtrTwoSame[iDistThreeExtrTwoSame,:3])
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,6] = distThreeExtrTwoSame[iDistThreeExtrTwoSame,5] > expt_freq_seq_mid
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,7] = None
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,8] = None
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,9] = sum(trial_tones[i_trial]>expt_freq_seq_mid)>1
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,10] = np.max(trial_tones[i_trial,:]) 
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,11] = distThreeExtrTwoSame[iDistThreeExtrTwoSame,10] > expt_freq_seq_mid 
            #print(distThreeExtrTwoSame[iDistThreeExtrTwoSame,[0,1,2,3,4,5,6,7,8,9]])
            distThreeExtrTwoSame[iDistThreeExtrTwoSame,12] = int(i_trial)
            iDistThreeExtrTwoSame += 1   

    for i_trial in range(len(trial_tones)):         
        if ((int(low_dist[0]-2*low_dist[1]<trial_tones[i_trial][0]<expt_freq_seq_mid) +
             int(low_dist[0]-2*low_dist[1]<trial_tones[i_trial][1]<expt_freq_seq_mid) +
             int(low_dist[0]-2*low_dist[1]<trial_tones[i_trial][2]<expt_freq_seq_mid))==2):
            oneDistractor[iOneDistractor,:3] = trial_tones[i_trial,:]
            oneDistractor[iOneDistractor,3] = trial_behaviour[i_trial]
            oneDistractor[iOneDistractor,4] = corrans[i_trial]
            iOneDistractor += 1   
            
        elif ((int(expt_freq_seq_mid<trial_tones[i_trial][0]<high_dist[0]+2*high_dist[1]) +
             int(expt_freq_seq_mid<trial_tones[i_trial][1]<high_dist[0]+2*high_dist[1]) +
             int(expt_freq_seq_mid<trial_tones[i_trial][2]<high_dist[0]+2*high_dist[1]))==2):
            oneDistractor[iOneDistractor,:3] = trial_tones[i_trial,:]
            oneDistractor[iOneDistractor,3] = trial_behaviour[i_trial]
            oneDistractor[iOneDistractor,4] = corrans[i_trial]
            iOneDistractor += 1
            
        elif ((int(low_dist[0]-2*low_dist[1]<trial_tones[i_trial][0]<high_dist[0]+2*high_dist[1]) +
             int(low_dist[0]-2*low_dist[1]<trial_tones[i_trial][1]<high_dist[0]+2*high_dist[1]) +
             int(low_dist[0]-2*low_dist[1]<trial_tones[i_trial][2]<high_dist[0]+2*high_dist[1]))==1):
            twoDistractors[iTwoDistractors,:3] = trial_tones[i_trial,:]
            twoDistractors[iTwoDistractors,3] = trial_behaviour[i_trial]
            twoDistractors[iTwoDistractors,4] = corrans[i_trial]
            iTwoDistractors += 1
    
    return(allGaussLow[:iAllGaussLow,:], 
           allGaussHigh[:iAllGaussHigh,:], 
           distGaussLow[:iDistGaussLow,:],
           distGaussHigh[:iDistGaussHigh,:],
           distLowOtherLow[:iDistLowOtherLow,:], 
           distHighOtherHigh[:iDistHighOtherHigh,:],
           distLowOtherHigh[:iDistLowOtherHigh,:], 
           distHighOtherLow[:iDistHighOtherLow,:],
           distLowOtherLowHigh[:iDistLowOtherLowHigh,:], 
           distHighOtherLowHigh[:iDistHighOtherLowHigh,:],
           distTwoExtrOtherGauss[:iDistTwoExtrOtherGauss,:], 
           distTwoExtrSameGauss[:iDistTwoExtrSameGauss,:],
           distThreeExtr[:iDistThreeExtr,:], 
           distThreeExtrTwoSame[:iDistThreeExtrTwoSame,:],
           oneDistractor[:iOneDistractor,:],
           twoDistractors[:iTwoDistractors,:])


In [ ]:
def accuracies(allGaussLow, allGaussHigh, distGaussLow, distGaussHigh, distLowOtherLow, 
               distHighOtherHigh, distLowOtherHigh, distHighOtherLow, distLowOtherLowHigh,
               distHighOtherLowHigh, distTwoExtrOtherGauss, distTwoExtrSameGauss, 
               distThreeExtrTwoSame, corrans_expt, trial_behaviour_expt):
    """
    Calculating accuracy table

    rows: different categories
    columns: different strategies - |ideal subject accuracy|% of trials that can be explained using a given strategy| *
    number of strategies
    """

    accuracy = np.zeros((10, 9))
    distSameOtherSame = np.concatenate((distLowOtherLow,distHighOtherHigh),axis=0)
    distractorGaussian = np.concatenate((distGaussLow,distGaussHigh),axis=0)
    allGaussian = np.concatenate((allGaussLow,allGaussHigh),axis=0)
    biasWithUnclearSignal = np.zeros((1,4))
    
    """
    accuracy[0,0] = np.mean(allGaussLow[:,3]==allGaussLow[:,4])
    accuracy[0,1] = np.mean(allGaussLow[:,4]==allGaussLow[:,6])
    accuracy[0,2] = np.mean(allGaussLow[:,3]==allGaussLow[:,6])
    accuracy[0,3] = np.mean(allGaussLow[:,4]==allGaussLow[:,8])
    accuracy[0,4] = np.mean(allGaussLow[:,3]==allGaussLow[:,8])
    accuracy[0,5] = np.mean(allGaussLow[:,4]==allGaussLow[:,9])
    accuracy[0,6] = np.mean(allGaussLow[:,3]==allGaussLow[:,9])
    accuracy[0,7] = np.mean(allGaussLow[:,4]==allGaussLow[:,11])
    accuracy[0,8] = np.mean(allGaussLow[:,3]==allGaussLow[:,11])
    
    accuracy[1,0] = np.mean(allGaussHigh[:,3]==allGaussHigh[:,4])
    accuracy[1,1] = np.mean(allGaussHigh[:,4]==allGaussHigh[:,6])
    accuracy[1,2] = np.mean(allGaussHigh[:,3]==allGaussHigh[:,6])
    accuracy[1,3] = np.mean(allGaussHigh[:,4]==allGaussHigh[:,8])
    accuracy[1,4] = np.mean(allGaussHigh[:,3]==allGaussHigh[:,8])
    accuracy[1,5] = np.mean(allGaussHigh[:,4]==allGaussHigh[:,9])
    accuracy[1,6] = np.mean(allGaussHigh[:,3]==allGaussHigh[:,9])
    accuracy[1,7] = np.mean(allGaussHigh[:,4]==allGaussHigh[:,11])
    accuracy[1,8] = np.mean(allGaussHigh[:,3]==allGaussHigh[:,11])
    """
    accuracy[0,0] = np.mean(allGaussian[:,3]==allGaussian[:,4])
    accuracy[0,1] = np.mean(allGaussian[:,4]==allGaussian[:,6])
    accuracy[0,2] = np.mean(allGaussian[:,3]==allGaussian[:,6])
    accuracy[0,3] = np.mean(allGaussian[:,4]==allGaussian[:,8])
    accuracy[0,4] = np.mean(allGaussian[:,3]==allGaussian[:,8])
    accuracy[0,5] = np.mean(allGaussian[:,4]==allGaussian[:,9])
    accuracy[0,6] = np.mean(allGaussian[:,3]==allGaussian[:,9])
    accuracy[0,7] = np.mean(allGaussian[:,4]==allGaussian[:,11])
    accuracy[0,8] = np.mean(allGaussian[:,3]==allGaussian[:,11])
    
    accuracy[2,0] = np.mean(distractorGaussian[:,3]==distractorGaussian[:,4])
    accuracy[2,1] = np.mean(distractorGaussian[:,4]==distractorGaussian[:,6])
    accuracy[2,2] = np.mean(distractorGaussian[:,3]==distractorGaussian[:,6])
    accuracy[2,3] = np.mean(distractorGaussian[:,4]==distractorGaussian[:,8])
    accuracy[2,4] = np.mean(distractorGaussian[:,3]==distractorGaussian[:,8])
    accuracy[2,5] = np.mean(distractorGaussian[:,4]==distractorGaussian[:,9])
    accuracy[2,6] = np.mean(distractorGaussian[:,3]==distractorGaussian[:,9])
    accuracy[2,7] = np.mean(distractorGaussian[:,4]==distractorGaussian[:,11])
    accuracy[2,8] = np.mean(distractorGaussian[:,3]==distractorGaussian[:,11])
    """
    
    accuracy[2,0] = np.mean(distGaussLow[:,3]==distGaussLow[:,4])
    accuracy[2,1] = np.mean(distGaussLow[:,4]==distGaussLow[:,6])
    accuracy[2,2] = np.mean(distGaussLow[:,3]==distGaussLow[:,6])
    accuracy[2,3] = np.mean(distGaussLow[:,4]==distGaussLow[:,8])
    accuracy[2,4] = np.mean(distGaussLow[:,3]==distGaussLow[:,8])
    accuracy[2,5] = np.mean(distGaussLow[:,4]==distGaussLow[:,9])
    accuracy[2,6] = np.mean(distGaussLow[:,3]==distGaussLow[:,9])
    accuracy[2,7] = np.mean(distGaussLow[:,4]==distGaussLow[:,11])
    accuracy[2,8] = np.mean(distGaussLow[:,3]==distGaussLow[:,11])
    
    accuracy[3,0] = np.mean(distGaussHigh[:,3]==distGaussHigh[:,4])
    accuracy[3,1] = np.mean(distGaussHigh[:,4]==distGaussHigh[:,6])
    accuracy[3,2] = np.mean(distGaussHigh[:,3]==distGaussHigh[:,6])
    accuracy[3,3] = np.mean(distGaussHigh[:,4]==distGaussHigh[:,8])
    accuracy[3,4] = np.mean(distGaussHigh[:,3]==distGaussHigh[:,8])
    accuracy[3,5] = np.mean(distGaussHigh[:,4]==distGaussHigh[:,9])
    accuracy[3,6] = np.mean(distGaussHigh[:,3]==distGaussHigh[:,9])
    accuracy[3,7] = np.mean(distGaussHigh[:,4]==distGaussHigh[:,11])
    accuracy[3,8] = np.mean(distGaussHigh[:,3]==distGaussHigh[:,11])

    accuracy[4,0] = np.mean(distLowOtherLow[:,3]==distLowOtherLow[:,4])
    accuracy[4,1] = np.mean(distLowOtherLow[:,4]==distLowOtherLow[:,6])
    accuracy[4,2] = np.mean(distLowOtherLow[:,3]==distLowOtherLow[:,6])
    accuracy[4,3] = np.mean(distLowOtherLow[:,4]==distLowOtherLow[:,8])
    accuracy[4,4] = np.mean(distLowOtherLow[:,3]==distLowOtherLow[:,8])
    accuracy[4,5] = np.mean(distLowOtherLow[:,4]==distLowOtherLow[:,9])
    accuracy[4,6] = np.mean(distLowOtherLow[:,3]==distLowOtherLow[:,9])
    accuracy[4,7] = np.mean(distLowOtherLow[:,4]==distLowOtherLow[:,11])
    accuracy[4,8] = np.mean(distLowOtherLow[:,3]==distLowOtherLow[:,11])

    accuracy[5,0] = np.mean(distHighOtherHigh[:,3]==distHighOtherHigh[:,4])
    accuracy[5,1] = np.mean(distHighOtherHigh[:,4]==distHighOtherHigh[:,6])
    accuracy[5,2] = np.mean(distHighOtherHigh[:,3]==distHighOtherHigh[:,6])
    accuracy[5,3] = np.mean(distHighOtherHigh[:,4]==distHighOtherHigh[:,8])
    accuracy[5,4] = np.mean(distHighOtherHigh[:,3]==distHighOtherHigh[:,8])
    accuracy[5,5] = np.mean(distHighOtherHigh[:,4]==distHighOtherHigh[:,9])
    accuracy[5,6] = np.mean(distHighOtherHigh[:,3]==distHighOtherHigh[:,9])
    accuracy[5,7] = np.mean(distHighOtherHigh[:,4]==distHighOtherHigh[:,11])
    accuracy[5,8] = np.mean(distHighOtherHigh[:,3]==distHighOtherHigh[:,11])
    """
    accuracy[3,0] = np.mean(distSameOtherSame[:,3]==distSameOtherSame[:,4])
    accuracy[3,1] = np.mean(distSameOtherSame[:,4]==distSameOtherSame[:,6])
    accuracy[3,2] = np.mean(distSameOtherSame[:,3]==distSameOtherSame[:,6])
    accuracy[3,3] = np.mean(distSameOtherSame[:,4]==distSameOtherSame[:,8])
    accuracy[3,4] = np.mean(distSameOtherSame[:,3]==distSameOtherSame[:,8])
    accuracy[3,5] = np.mean(distSameOtherSame[:,4]==distSameOtherSame[:,9])
    accuracy[3,6] = np.mean(distSameOtherSame[:,3]==distSameOtherSame[:,9])
    accuracy[3,7] = np.mean(distSameOtherSame[:,4]==distSameOtherSame[:,11])
    accuracy[3,8] = np.mean(distSameOtherSame[:,3]==distSameOtherSame[:,11])
    
    accuracy[4,0] = np.mean(distLowOtherHigh[:,3]==distLowOtherHigh[:,4])
    accuracy[4,1] = np.mean(distLowOtherHigh[:,4]==distLowOtherHigh[:,6])
    accuracy[4,2] = np.mean(distLowOtherHigh[:,3]==distLowOtherHigh[:,6])
    accuracy[4,3] = np.mean(distLowOtherHigh[:,4]==distLowOtherHigh[:,8])
    accuracy[4,4] = np.mean(distLowOtherHigh[:,3]==distLowOtherHigh[:,8])
    accuracy[4,5] = np.mean(distLowOtherHigh[:,4]==distLowOtherHigh[:,9])
    accuracy[4,6] = np.mean(distLowOtherHigh[:,3]==distLowOtherHigh[:,9])
    accuracy[4,7] = np.mean(distLowOtherHigh[:,4]==distLowOtherHigh[:,11])
    accuracy[4,8] = np.mean(distLowOtherHigh[:,3]==distLowOtherHigh[:,11])

    accuracy[5,0] = np.mean(distHighOtherLow[:,3]==distHighOtherLow[:,4])
    accuracy[5,1] = np.mean(distHighOtherLow[:,4]==distHighOtherLow[:,6])
    accuracy[5,2] = np.mean(distHighOtherLow[:,3]==distHighOtherLow[:,6])
    accuracy[5,3] = np.mean(distHighOtherLow[:,4]==distHighOtherLow[:,8])
    accuracy[5,4] = np.mean(distHighOtherLow[:,3]==distHighOtherLow[:,8])
    accuracy[5,5] = np.mean(distHighOtherLow[:,4]==distHighOtherLow[:,9])
    accuracy[5,6] = np.mean(distHighOtherLow[:,3]==distHighOtherLow[:,9])
    accuracy[5,7] = np.mean(distHighOtherLow[:,4]==distHighOtherLow[:,11])
    accuracy[5,8] = np.mean(distHighOtherLow[:,3]==distHighOtherLow[:,11])

    accuracy[6,0] = np.mean(distTwoExtrOtherGauss[:,3]==distTwoExtrOtherGauss[:,4])
    accuracy[6,1] = np.mean(distTwoExtrOtherGauss[:,4]==distTwoExtrOtherGauss[:,6])
    accuracy[6,2] = np.mean(distTwoExtrOtherGauss[:,3]==distTwoExtrOtherGauss[:,6])
    accuracy[6,3] = np.mean(distTwoExtrOtherGauss[:,4]==distTwoExtrOtherGauss[:,8])
    accuracy[6,4] = np.mean(distTwoExtrOtherGauss[:,3]==distTwoExtrOtherGauss[:,8])
    accuracy[6,5] = np.mean(distTwoExtrOtherGauss[:,4]==distTwoExtrOtherGauss[:,9])
    accuracy[6,6] = np.mean(distTwoExtrOtherGauss[:,3]==distTwoExtrOtherGauss[:,9])
    accuracy[6,7] = np.mean(distTwoExtrOtherGauss[:,4]==distTwoExtrOtherGauss[:,11])
    accuracy[6,8] = np.mean(distTwoExtrOtherGauss[:,3]==distTwoExtrOtherGauss[:,11])

    accuracy[7,0] = np.mean(distTwoExtrSameGauss[:,3]==distTwoExtrSameGauss[:,4])
    accuracy[7,1] = np.mean(distTwoExtrSameGauss[:,4]==distTwoExtrSameGauss[:,6])
    accuracy[7,2] = np.mean(distTwoExtrSameGauss[:,3]==distTwoExtrSameGauss[:,6])
    accuracy[7,3] = np.mean(distTwoExtrSameGauss[:,4]==distTwoExtrSameGauss[:,8])
    accuracy[7,4] = np.mean(distTwoExtrSameGauss[:,3]==distTwoExtrSameGauss[:,8])
    accuracy[7,5] = np.mean(distTwoExtrSameGauss[:,4]==distTwoExtrSameGauss[:,9])
    accuracy[7,6] = np.mean(distTwoExtrSameGauss[:,3]==distTwoExtrSameGauss[:,9])
    accuracy[7,7] = np.mean(distTwoExtrSameGauss[:,4]==distTwoExtrSameGauss[:,11])
    accuracy[7,8] = np.mean(distTwoExtrSameGauss[:,3]==distTwoExtrSameGauss[:,11])
    """
    accuracy[8,0] = np.mean(distThreeExtr[:,3]==distThreeExtr[:,4])
    accuracy[8,1] = np.mean(distThreeExtr[:,4]==distThreeExtr[:,6])
    accuracy[8,2] = np.mean(distThreeExtr[:,3]==distThreeExtr[:,6])
    accuracy[8,3] = np.mean(distThreeExtr[:,4]==distThreeExtr[:,8])
    accuracy[8,4] = np.mean(distThreeExtr[:,3]==distThreeExtr[:,8])
    accuracy[8,5] = np.mean(distThreeExtr[:,4]==distThreeExtr[:,9])
    accuracy[8,6] = np.mean(distThreeExtr[:,3]==distThreeExtr[:,9])
    accuracy[8,7] = np.mean(distThreeExtr[:,4]==distThreeExtr[:,11])
    accuracy[8,8] = np.mean(distThreeExtr[:,3]==distThreeExtr[:,11])
    """
    accuracy[9,0] = np.mean(distThreeExtrTwoSame[:,3]==distThreeExtrTwoSame[:,4])
    accuracy[9,1] = np.mean(distThreeExtrTwoSame[:,4]==distThreeExtrTwoSame[:,6])
    accuracy[9,2] = np.mean(distThreeExtrTwoSame[:,3]==distThreeExtrTwoSame[:,6])
    accuracy[9,3] = np.mean(distThreeExtrTwoSame[:,4]==distThreeExtrTwoSame[:,8])
    accuracy[9,4] = np.mean(distThreeExtrTwoSame[:,3]==distThreeExtrTwoSame[:,8])
    accuracy[9,5] = np.mean(distThreeExtrTwoSame[:,4]==distThreeExtrTwoSame[:,9])
    accuracy[9,6] = np.mean(distThreeExtrTwoSame[:,3]==distThreeExtrTwoSame[:,9])
    accuracy[9,7] = np.mean(distThreeExtrTwoSame[:,4]==distThreeExtrTwoSame[:,11])
    accuracy[9,8] = np.mean(distThreeExtrTwoSame[:,3]==distThreeExtrTwoSame[:,11])
    
    biasWithUnclearSignal[:,0] = np.mean(distLowOtherLowHigh[:,3]==0)
    biasWithUnclearSignal[:,1] = np.mean(distLowOtherLowHigh[:,3]==1)
    biasWithUnclearSignal[:,2] = np.mean(distHighOtherLowHigh[:,3]==0)
    biasWithUnclearSignal[:,3] = np.mean(distHighOtherLowHigh[:,3]==1)
    
    print("Total accuracy of the subject: ")
    print(np.sum(corrans_expt==trial_behaviour_expt)/len(trial_behaviour_expt))

    print("Overall accuracy of categories: ")
    print(accuracy[:,0])

    print("optimal subject accuracies: ")
    print(accuracy[:,[1,3,5,7]])

    print("what percentage of the data can be explained by the optimal subject: ")
    print(accuracy[:,[2,4,6,8]])   
    
    return accuracy*100, biasWithUnclearSignal*100


In [ ]:

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5ea1f4fa7a70090fd0715b34_categorization_task_2021-03-01_18h06.17.396.csv');

trial_tones_5b34, trial_behaviour_5b34, trial_tone_cat_5b34, corrans_5b34 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5fad0bb914cb0035f917d619_categorization_task_2021-03-01_17h13.10.896.csv');

trial_tones_d619, trial_behaviour_d619, trial_tone_cat_d619, corrans_d619 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5cc86a4e264a60000124c2e0_categorization_task_2021-03-01_18h07.47.350.csv');

trial_tones_c2e0, trial_behaviour_c2e0, trial_tone_cat_c2e0, corrans_c2e0 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5f3990fea9ef865370ac735a_categorization_task_2021-03-01_17h41.13.928.csv');

trial_tones_735a, trial_behaviour_735a, trial_tone_cat_735a, corrans_735a = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5f3801b18c88962be7831304_categorization_task_2021-03-01_19h52.16.006.csv');

trial_tones_1304, trial_behaviour_1304, trial_tone_cat_1304, corrans_1304 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/60143ceed282ae3ceda28de3_categorization_task_2021-03-01_18h53.15.963.csv');

trial_tones_8de3, trial_behaviour_8de3, trial_tone_cat_8de3, corrans_8de3 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5ec48aac33b93f000ab96b7f_categorization_task_2021-03-01_20h56.07.985.csv');

trial_tones_6b7f, trial_behaviour_6b7f, trial_tone_cat_6b7f, corrans_6b7f = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5ec3a28f46bca2016fc91c3f_categorization_task_2021-03-01_21h03.59.423.csv');

trial_tones_1c3f, trial_behaviour_1c3f, trial_tone_cat_1c3f, corrans_1c3f = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5ec3a1fe1bfa1720b3731604_categorization_task_2021-03-01_21h58.03.550.csv');

trial_tones_1604, trial_behaviour_1604, trial_tone_cat_1604, corrans_1604 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5f14e794a89ac35a3f89801d_categorization_task_2021-02-19_23h35.01.131.csv');

trial_tones_801d, trial_behaviour_801d, trial_tone_cat_801d, corrans_801d = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5f4b1ba295b8186ec4901396_categorization_task_2021-02-19_18h32.43.841.csv');

trial_tones_1396, trial_behaviour_1396, trial_tone_cat_1396, corrans_1396 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/5f567ed454a3b50b805354db_categorization_task_2021-02-19_22h29.09.897.csv');

trial_tones_54db, trial_behaviour_54db, trial_tone_cat_54db, corrans_54db = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-03-03_04h34.17_b73c4368-7bd9-11eb-a333-ac1f6b405aea/VIOLINIST_categorization_task_2021-03-02_18h00.06.158.csv');

trial_tones_viol, trial_behaviour_viol, trial_tone_cat_viol, corrans_viol = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-04-18_21h32.52_a165fbe6-a08d-11eb-a749-ac1f6b405aea/603ecb8767d4bc041eedb4c7_categorization_task_2021-04-18_12h12.29.327.csv');

trial_tones_b4c7, trial_behaviour_b4c7, trial_tone_cat_b4c7, corrans_b4c7 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-04-19_13h07.33_346fb076-a110-11eb-a757-ac1f6b405aea/a12e_categorization_task_2021-04-19_11h01.01.358.csv');

trial_tones_a12e, trial_behaviour_a12e, trial_tone_cat_a12e, corrans_a12e = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-16_23h33.50_2b5881da-b69f-11eb-a977-ac1f6b405aea/6060687bbc223f086e4fc653_categorization_task_2021-05-16_21h19.38.241.csv');

trial_tones_c653, trial_behaviour_c653, trial_tone_cat_c653, corrans_c653 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-16_23h33.50_2b5881da-b69f-11eb-a977-ac1f6b405aea/607950e0efb144952e6f4b7f_categorization_task_2021-05-16_11h00.32.119.csv');

trial_tones_4b7f, trial_behaviour_4b7f, trial_tone_cat_4b7f, corrans_4b7f = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-16_23h33.50_2b5881da-b69f-11eb-a977-ac1f6b405aea/609feb16aafd2f71e730fc3a_categorization_task_2021-05-16_15h37.20.773.csv');

trial_tones_fc3a, trial_behaviour_fc3a, trial_tone_cat_fc3a, corrans_fc3a = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/601306071a72651244570d04_categorization_task_2021-05-18_04h46.56.555.csv');

trial_tones_0d04, trial_behaviour_0d04, trial_tone_cat_0d04, corrans_0d04 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/609dd2bc3360f6e164901570_categorization_task_2021-05-17_22h53.33.874.csv');

trial_tones_1570, trial_behaviour_1570, trial_tone_cat_1570, corrans_1570 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5e830b929f2ed2018ec42099_categorization_task_2021-05-17_22h01.23.634.csv');

trial_tones_2099, trial_behaviour_2099, trial_tone_cat_2099, corrans_2099 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5f627674b6f93f19ff74592d_categorization_task_2021-05-17_22h05.20.424.csv');

trial_tones_592d, trial_behaviour_592d, trial_tone_cat_592d, corrans_592d = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/606030fbea6f7e84bc7aa45d_categorization_task_2021-05-17_20h04.48.021.csv');

trial_tones_a45d, trial_behaviour_a45d, trial_tone_cat_a45d, corrans_a45d = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5f3cb7486048540009840cf1_categorization_task_2021-05-18_05h15.37.577.csv');

trial_tones_0cf1, trial_behaviour_0cf1, trial_tone_cat_0cf1, corrans_0cf1 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/6062c088821c76a49374e453_categorization_task_2021-05-17_23h24.18.467.csv');

trial_tones_e453, trial_behaviour_e453, trial_tone_cat_e453, corrans_e453 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/60660870280f2b522cc3214b_categorization_task_2021-05-18_09h04.16.970.csv');

trial_tones_214b, trial_behaviour_214b, trial_tone_cat_214b, corrans_214b = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5f3716c55fa34618ff41e045_categorization_task_2021-05-18_10h25.52.803.csv');

trial_tones_e045, trial_behaviour_e045, trial_tone_cat_e045, corrans_e045 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5f2a12f5837dd81e5881b7cc_categorization_task_2021-05-18_16h19.05.932.csv');

trial_tones_b7cc, trial_behaviour_b7cc, trial_tone_cat_b7cc, corrans_b7cc = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5f591c6d75be841230aa6ced_categorization_task_2021-05-19_00h36.07.372.csv');

trial_tones_6ced, trial_behaviour_6ced, trial_tone_cat_6ced, corrans_6ced = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-19_20h09.38_23b63e9a-b8de-11eb-a9b5-ac1f6b405aea/5f4032479b2b92152488188a_categorization_task_2021-05-19_16h44.41.870.csv');

trial_tones_188a, trial_behaviour_188a, trial_tone_cat_188a, corrans_188a = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-20_20h09.05_3a409060-b9a7-11eb-a9da-ac1f6b405aea/5e485d1d4edf63000d0b1620_categorization_task_2021-05-19_19h19.38.331.csv');

trial_tones_1620, trial_behaviour_1620, trial_tone_cat_1620, corrans_1620 = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-20_20h09.05_3a409060-b9a7-11eb-a9da-ac1f6b405aea/60a3c37fb7414cf62b79e21e_categorization_task_2021-05-20_14h16.08.213.csv');

trial_tones_e12e, trial_behaviour_e12e, trial_tone_cat_e12e, corrans_e12e = analysis2(csv_test,csv_data)

csv_test = pd.read_csv('../auditory_categorization_prolific/important_things_not_included_in_assets/allTrials.csv')
csv_data = pd.read_csv('auditory_categorization_prolific_online_data/human_auditory_categorization_91686_2021-05-20_20h09.05_3a409060-b9a7-11eb-a9da-ac1f6b405aea/5e7235e6d302e91679226b0e_categorization_task_2021-05-20_16h24.10.294.csv');

trial_tones_6b0e, trial_behaviour_6b0e, trial_tone_cat_6b0e, corrans_6b0e = analysis2(csv_test,csv_data)

In [ ]:
csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5fad0bb914cb0035f917d619_categorization_task_longLow_2021-03-29_23h07.31.656.csv');

trial_tones_d619Lc, trial_behaviour_d619Lc, trial_tone_cat_d619Lc, corrans_d619Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5cc86a4e264a60000124c2e0_categorization_task_longLow_2021-04-01_10h42.12.417.csv');

trial_tones_c2e0Lc, trial_behaviour_c2e0Lc, trial_tone_cat_c2e0Lc, corrans_c2e0Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-05-13_18h12.29_c75655f2-b416-11eb-a951-ac1f6b405aea/5f3990fea9ef865370ac735a_categorization_task_longLow_2021-04-19_17h09.03.462.csv');

trial_tones_735aLc, trial_behaviour_735aLc, trial_tone_cat_735aLc, corrans_735aLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-05-15_18h46.10_d0fb81d4-b5ad-11eb-a96f-ac1f6b405aea/5f3801b18c88962be7831304_categorization_task_longLow_2021-05-15_19h10.15.555.csv');

trial_tones_1304Lc, trial_behaviour_1304Lc, trial_tone_cat_1304Lc, corrans_1304Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5ec48aac33b93f000ab96b7f_categorization_task_longLow_2021-03-30_00h40.03.108.csv');

trial_tones_6b7fLc, trial_behaviour_6b7fLc, trial_tone_cat_6b7fLc, corrans_6b7fLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5ec3a28f46bca2016fc91c3f_categorization_task_longLow_2021-03-30_01h15.32.575.csv');

trial_tones_1c3fLc, trial_behaviour_1c3fLc, trial_tone_cat_1c3fLc, corrans_1c3fLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5ec3a1fe1bfa1720b3731604_categorization_task_longLow_2021-03-30_00h40.53.029.csv');

trial_tones_1604Lc, trial_behaviour_1604Lc, trial_tone_cat_1604Lc, corrans_1604Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5f14e794a89ac35a3f89801d_categorization_task_longLow_2021-05-17_15h46.25.182.csv');

trial_tones_801dLc, trial_behaviour_801dLc, trial_tone_cat_801dLc, corrans_801dLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5f4b1ba295b8186ec4901396_categorization_task_longLow_2021-03-31_09h58.35.793.csv');

trial_tones_1396Lc, trial_behaviour_1396Lc, trial_tone_cat_1396Lc, corrans_1396Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/603ecb8767d4bc041eedb4c7_categorization_task_longLow_2021-03-29_23h42.07.895.csv');

trial_tones_b4c7Lc, trial_behaviour_b4c7Lc, trial_tone_cat_b4c7Lc, corrans_b4c7Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-04-01_21h21.21_349e7548-9330-11eb-a617-ac1f6b405aea/5f809cdbe0804f0ec6daa12e_categorization_task_longLow_2021-03-30_10h54.28.163.csv');

trial_tones_a12eLc, trial_behaviour_a12eLc, trial_tone_cat_a12eLc, corrans_a12eLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-05-15_18h46.10_d0fb81d4-b5ad-11eb-a96f-ac1f6b405aea/6060687bbc223f086e4fc653_categorization_task_longLow_2021-05-15_20h38.22.974.csv');

trial_tones_c653Lc, trial_behaviour_c653Lc, trial_tone_cat_c653Lc, corrans_c653Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-05-15_18h46.10_d0fb81d4-b5ad-11eb-a96f-ac1f6b405aea/607950e0efb144952e6f4b7f_categorization_task_longLow_2021-05-15_19h42.31.771.csv');

trial_tones_4b7fLc, trial_behaviour_4b7fLc, trial_tone_cat_4b7fLc, corrans_4b7fLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-05-15_18h46.10_d0fb81d4-b5ad-11eb-a96f-ac1f6b405aea/609feb16aafd2f71e730fc3a_categorization_task_longLow_2021-05-15_20h37.38.994.csv');

trial_tones_fc3aLc, trial_behaviour_fc3aLc, trial_tone_cat_fc3aLc, corrans_fc3aLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/601306071a72651244570d04_categorization_task_longLow_2021-06-09_03h41.33.989.csv');

trial_tones_0d04Lc, trial_behaviour_0d04Lc, trial_tone_cat_0d04Lc, corrans_0d04Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-11_19h04.17_d1d63a32-cae7-11eb-abb1-ac1f6b405aea/609dd2bc3360f6e164901570_categorization_task_longLow_2021-06-11_13h42.28.848.csv');

trial_tones_1570Lc, trial_behaviour_1570Lc, trial_tone_cat_1570Lc, corrans_1570Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5e830b929f2ed2018ec42099_categorization_task_longLow_2021-06-08_14h09.42.717.csv');

trial_tones_2099Lc, trial_behaviour_2099Lc, trial_tone_cat_2099Lc, corrans_2099Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5f627674b6f93f19ff74592d_categorization_task_longLow_2021-06-08_12h00.15.358.csv');

trial_tones_592dLc, trial_behaviour_592dLc, trial_tone_cat_592dLc, corrans_592dLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/606030fbea6f7e84bc7aa45d_categorization_task_longLow_2021-06-08_13h36.49.749.csv');

trial_tones_a45dLc, trial_behaviour_a45dLc, trial_tone_cat_a45dLc, corrans_a45dLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5f3cb7486048540009840cf1_categorization_task_longLow_2021-06-08_21h40.18.046.csv');

trial_tones_0cf1Lc, trial_behaviour_0cf1Lc, trial_tone_cat_0cf1Lc, corrans_0cf1Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/6062c088821c76a49374e453_categorization_task_longLow_2021-06-08_17h23.26.803.csv');

trial_tones_e453Lc, trial_behaviour_e453Lc, trial_tone_cat_e453Lc, corrans_e453Lc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/60660870280f2b522cc3214b_categorization_task_longLow_2021-06-08_10h56.05.372.csv');

trial_tones_214bLc, trial_behaviour_214bLc, trial_tone_cat_214bLc, corrans_214bLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5f2a12f5837dd81e5881b7cc_categorization_task_longLow_2021-06-08_18h38.21.637.csv');

trial_tones_b7ccLc, trial_behaviour_b7ccLc, trial_tone_cat_b7ccLc, corrans_b7ccLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5f591c6d75be841230aa6ced_categorization_task_longLow_2021-06-08_17h43.28.458.csv');

trial_tones_6cedLc, trial_behaviour_6cedLc, trial_tone_cat_6cedLc, corrans_6cedLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/5f4032479b2b92152488188a_categorization_task_longLow_2021-06-08_18h13.56.450.csv');

trial_tones_188aLc, trial_behaviour_188aLc, trial_tone_cat_188aLc, corrans_188aLc = analysis2(csv_testLc,csv_dataLc)

csv_testLc = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv')
csv_dataLc = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-10_00h51.07_f0ebe880-c985-11eb-ab99-ac1f6b405aea/60a3c37fb7414cf62b79e21e_categorization_task_longLow_2021-06-08_20h10.04.207.csv');

trial_tones_e12eLc, trial_behaviour_e12eLc, trial_tone_cat_e12eLc, corrans_e12eLc = analysis2(csv_testLc,csv_dataLc)


In [ ]:
csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5f3990fea9ef865370ac735a_categorization_task_longHigh_2021-06-18_15h49.59.982.csv');

trial_tones_735aHc, trial_behaviour_735aHc, trial_tone_cat_735aHc, corrans_735aHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5f3801b18c88962be7831304_categorization_task_longHigh_2021-06-17_14h28.57.096.csv');

trial_tones_1304Hc, trial_behaviour_1304Hc, trial_tone_cat_1304Hc, corrans_1304Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_21h01.46_8ea3ab86-d141-11eb-abf1-ac1f6b405aea/5ec48aac33b93f000ab96b7f_categorization_task_longHigh_2021-06-19_21h42.04.614.csv');

trial_tones_6b7fHc, trial_behaviour_6b7fHc, trial_tone_cat_6b7fHc, corrans_6b7fHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5ec3a28f46bca2016fc91c3f_categorization_task_longHigh_2021-06-16_10h08.17.055.csv');

trial_tones_1c3fHc, trial_behaviour_1c3fHc, trial_tone_cat_1c3fHc, corrans_1c3fHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5ec3a1fe1bfa1720b3731604_categorization_task_longHigh_2021-06-17_12h28.40.530.csv');

trial_tones_1604Hc, trial_behaviour_1604Hc, trial_tone_cat_1604Hc, corrans_1604Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-20_21h31.41_e74189a2-d20e-11eb-abf6-ac1f6b405aea/5f4b1ba295b8186ec4901396_categorization_task_longHigh_2021-06-20_14h09.48.106.csv');

trial_tones_1396Hc, trial_behaviour_1396Hc, trial_tone_cat_1396Hc, corrans_1396Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-20_21h31.41_e74189a2-d20e-11eb-abf6-ac1f6b405aea/603ecb8767d4bc041eedb4c7_categorization_task_longHigh_2021-06-20_21h45.30.847.csv');

trial_tones_b4c7Hc, trial_behaviour_b4c7Hc, trial_tone_cat_b4c7Hc, corrans_b4c7Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5f809cdbe0804f0ec6daa12e_categorization_task_longHigh_2021-06-16_09h31.02.317.csv');

trial_tones_a12eHc, trial_behaviour_a12eHc, trial_tone_cat_a12eHc, corrans_a12eHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/6060687bbc223f086e4fc653_categorization_task_longHigh_2021-06-16_15h06.31.093.csv');

trial_tones_c653Hc, trial_behaviour_c653Hc, trial_tone_cat_c653Hc, corrans_c653Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/607950e0efb144952e6f4b7f_categorization_task_longHigh_2021-06-19_11h32.49.411.csv')
trial_tones_4b7fHc, trial_behaviour_4b7fHc, trial_tone_cat_4b7fHc, corrans_4b7fHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/609feb16aafd2f71e730fc3a_categorization_task_longHigh_2021-06-16_09h05.25.383.csv');

trial_tones_fc3aHc, trial_behaviour_fc3aHc, trial_tone_cat_fc3aHc, corrans_fc3aHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/601306071a72651244570d04_categorization_task_longHigh_2021-06-16_23h31.15.482.csv');

trial_tones_0d04Hc, trial_behaviour_0d04Hc, trial_tone_cat_0d04Hc, corrans_0d04Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5e830b929f2ed2018ec42099_categorization_task_longHigh_2021-06-17_22h42.21.250.csv');

trial_tones_2099Hc, trial_behaviour_2099Hc, trial_tone_cat_2099Hc, corrans_2099Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/5f627674b6f93f19ff74592d_categorization_task_longHigh_2021-06-16_16h30.18.835.csv');

trial_tones_592dHc, trial_behaviour_592dHc, trial_tone_cat_592dHc, corrans_592dHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/606030fbea6f7e84bc7aa45d_categorization_task_longHigh_2021-06-16_05h39.27.990.csv');

trial_tones_a45dHc, trial_behaviour_a45dHc, trial_tone_cat_a45dHc, corrans_a45dHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/6062c088821c76a49374e453_categorization_task_longHigh_2021-06-16_00h53.08.975.csv');

trial_tones_e453Hc, trial_behaviour_e453Hc, trial_tone_cat_e453Hc, corrans_e453Hc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-06-19_15h40.10_a17ef1de-d114-11eb-abf0-ac1f6b405aea/60660870280f2b522cc3214b_categorization_task_longHigh_2021-06-16_14h49.32.751.csv');

trial_tones_214bHc, trial_behaviour_214bHc, trial_tone_cat_214bHc, corrans_214bHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-09-13_16h55.31_16f797ec-880d-4ffe-8c64-e679fe6e5c9b/5f2a12f5837dd81e5881b7cc_categorization_task_longHigh_2021-06-16_00h09.32.674.csv');

trial_tones_b7ccHc, trial_behaviour_b7ccHc, trial_tone_cat_b7ccHc, corrans_b7ccHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
DataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-09-13_16h55.31_16f797ec-880d-4ffe-8c64-e679fe6e5c9b/5f591c6d75be841230aa6ced_categorization_task_longHigh_2021-06-22_04h11.02.407.csv');                       
trial_tones_6cedHc, trial_behaviour_6cedHc, trial_tone_cat_6cedHc, corrans_6cedHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-09-13_16h55.31_16f797ec-880d-4ffe-8c64-e679fe6e5c9b/5f4032479b2b92152488188a_categorization_task_longHigh_2021-06-16_06h34.48.233.csv');

trial_tones_188aHc, trial_behaviour_188aHc, trial_tone_cat_188aHc, corrans_188aHc = analysis2(csv_testHc,csv_dataHc)

csv_testHc = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
csv_dataHc = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2021-09-13_16h55.31_16f797ec-880d-4ffe-8c64-e679fe6e5c9b/60a3c37fb7414cf62b79e21e_categorization_task_longHigh_2021-06-16_22h13.55.728.csv');

trial_tones_e12eHc, trial_behaviour_e12eHc, trial_tone_cat_e12eHc, corrans_e12eHc = analysis2(csv_testHc,csv_dataHc)


In [ ]:

log_freq_percept = np.arange(0.6,4.7,0.1) # array of possible perceptual tones
    
def plotting(allGaussian, distractorGaussian, distLowOtherHigh, distHighOtherLow):
        
    """
    1d plots
    """
    
    allMeanTone_behvSubjArray = np.zeros((4,len(logFreqBins)))
    GaussMeanTone_behvSubjArray = np.zeros((4,len(logFreqBins)))
    
    plt.figure()
    plt.xlabel('Mean of log10(freq) of all tones')
    plt.ylabel('p(B_H|T)')
    cnt=0
    for iCategory in [allGaussian, distractorGaussian, distLowOtherHigh, distHighOtherLow]:     
        allMeanLogBins = np.zeros((len(iCategory),))
        allMeanTone_behaviourOptimal = np.zeros((len(logFreqBins),))
        allMeanTone_behaviourSubj = np.zeros((len(logFreqBins),))    

        for iTrial in range(len(iCategory)):
            allMeanLogBins[iTrial] = logFreqBins[np.where(logFreqBins < iCategory[iTrial,5])[0][-1]]      
        for i_tone in range(len(logFreqBins)):
            allMeanTone_behaviourOptimal[i_tone] = np.mean(iCategory[allMeanLogBins==logFreqBins[i_tone],6])
            allMeanTone_behaviourSubj[i_tone] = np.mean(iCategory[allMeanLogBins==logFreqBins[i_tone],3])     
        #plt.plot(logFreqBins, allMeanTone_behaviourOptimal)
        plt.plot(logFreqBins, allMeanTone_behaviourSubj)  
        allMeanTone_behvSubjArray[cnt,:] = allMeanTone_behaviourSubj
        cnt += 1

    plt.figure()
    plt.xlabel('Mean of log10(freq) of Tones from Gaussians)')
    plt.ylabel('p(B_H|T)')
    cnt=0
    for iCategory in [allGaussian, distractorGaussian, distLowOtherHigh, distHighOtherLow]:        
        GaussMeanLogBins = np.zeros((len(iCategory),))
        GaussMeanTone_behaviourOptimal = np.zeros((len(logFreqBins),))
        GaussMeanTone_behaviourSubj = np.zeros((len(logFreqBins),))    

        for iTrial in range(len(iCategory)):
            GaussMeanLogBins[iTrial] = logFreqBins[np.where(logFreqBins < iCategory[iTrial,7])[0][-1]]   
        for i_tone in range(len(logFreqBins)):
            GaussMeanTone_behaviourOptimal[i_tone] = np.mean(iCategory[GaussMeanLogBins==logFreqBins[i_tone],8])
            GaussMeanTone_behaviourSubj[i_tone] = np.mean(iCategory[GaussMeanLogBins==logFreqBins[i_tone],3])     
        #plt.plot(logFreqBins, allMeanTone_behaviourOptimal)
        plt.plot(logFreqBins, GaussMeanTone_behaviourSubj)   
        GaussMeanTone_behvSubjArray[cnt,:] = GaussMeanTone_behaviourSubj
        cnt += 1
     
    return allMeanTone_behvSubjArray, GaussMeanTone_behvSubjArray

In [ ]:
"""
Ideal subject behaviour under all three strategies
"""
def idealSubjectBehaviour(tonesPresented):
    idealMeanBehaviour = (np.mean(tonesPresented,axis=1)>expt_freq_seq_mid).astype(int) 
    idealVotingBehaviour = (np.sum(tonesPresented>expt_freq_seq_mid,axis=1)>1).astype(int)
    idealSignalMeanBehaviour = np.copy(idealVotingBehaviour)
    for ibehv in range(len(idealSignalMeanBehaviour)):
        if (sum(tonesPresented[ibehv]>high_dist[0]+2*high_dist[1])
              + sum(tonesPresented[ibehv]<low_dist[0]-2*low_dist[1])==3):
            idealSignalMeanBehaviour[ibehv] = np.random.choice([0,1])
        elif (sum(tonesPresented[ibehv]>expt_freq_seq_mid)==2 
              and sum(tonesPresented[ibehv]<expt_freq_seq_mid)==1 
              and sum(tonesPresented[ibehv]>(high_dist[0]+2*high_dist[1]))==1
              and sum(tonesPresented[ibehv]<(low_dist[0]-2*low_dist[1]))==0):
            idealSignalMeanBehaviour[ibehv] = np.random.choice([0,1])
        elif (sum(tonesPresented[ibehv]>expt_freq_seq_mid)==1
              and sum(tonesPresented[ibehv]<expt_freq_seq_mid)==2 
              and sum(tonesPresented[ibehv]>(high_dist[0]+2*high_dist[1]))==0
              and sum(tonesPresented[ibehv]<(low_dist[0]-2*low_dist[1]))==1):
            idealSignalMeanBehaviour[ibehv] = np.random.choice([0,1]) 
        elif (sum(tonesPresented[ibehv]>(high_dist[0]+2*high_dist[1]))==0 
              and sum(tonesPresented[ibehv]>expt_freq_seq_mid)==1
              and sum(tonesPresented[ibehv]<(low_dist[0]-2*low_dist[1]))==2):
            idealSignalMeanBehaviour[ibehv] = 1 
        elif (sum(tonesPresented[ibehv]>(high_dist[0]+2*high_dist[1]))==2 
              and sum(tonesPresented[ibehv]<expt_freq_seq_mid)==1
              and sum(tonesPresented[ibehv]<(low_dist[0]-2*low_dist[1]))==0):  
            idealSignalMeanBehaviour[ibehv] = 0
    return idealMeanBehaviour, idealSignalMeanBehaviour, idealVotingBehaviour


def compareBehaviourWithDistractors(tonesPresented,subjectBehaviour,idealMeanBehaviour,
                                    idealSignalMeanBehaviour, idealVotingBehaviour):
    
    lowTrialsWithDistractors = np.array([],dtype=int)
    highTrialsWithDistractors = np.array([],dtype=int)
    for i_trial in range(len(tonesPresented)):
        if (sum(tonesPresented[i_trial]>(high_dist[0]+2*high_dist[1]))==0 
          and sum(tonesPresented[i_trial]>expt_freq_seq_mid)==1
          and sum(tonesPresented[i_trial]<(low_dist[0]-2*low_dist[1]))==1):
            highTrialsWithDistractors = np.append(highTrialsWithDistractors,i_trial)
        if (sum(tonesPresented[i_trial]>(high_dist[0]+2*high_dist[1]))==1 
          and sum(tonesPresented[i_trial]<expt_freq_seq_mid)==1
          and sum(tonesPresented[i_trial]<(low_dist[0]-2*low_dist[1]))==0):  
            lowTrialsWithDistractors = np.append(lowTrialsWithDistractors,i_trial)
    trialsWithDistractors = np.union1d(lowTrialsWithDistractors, highTrialsWithDistractors)
    accuracyMeanBehaviour = np.mean(subjectBehaviour[trialsWithDistractors]==
                                   idealMeanBehaviour[trialsWithDistractors])
    accuracySignalMeanBehaviuor = np.mean(subjectBehaviour[trialsWithDistractors]==
                                         idealSignalMeanBehaviour[trialsWithDistractors])
    accuracyVotingBehaviour = np.mean(subjectBehaviour[trialsWithDistractors]==
                                     idealVotingBehaviour[trialsWithDistractors])
    return accuracyMeanBehaviour, accuracySignalMeanBehaviuor, accuracyVotingBehaviour
    

In [ ]:
"""
Analyzing no context data
"""

trial_tones_expt = [trial_tones_5b34, trial_tones_d619, trial_tones_c2e0, trial_tones_735a, trial_tones_1304,
                    trial_tones_8de3, trial_tones_6b7f, trial_tones_1c3f, trial_tones_1604, trial_tones_801d,
                    trial_tones_1396, trial_tones_54db, trial_tones_viol, trial_tones_b4c7, trial_tones_a12e,
                    trial_tones_c653, trial_tones_4b7f, trial_tones_fc3a, trial_tones_0d04, trial_tones_1570,
                    trial_tones_2099, trial_tones_592d, trial_tones_a45d, trial_tones_0cf1, trial_tones_e453,
                    trial_tones_214b, trial_tones_e045, trial_tones_b7cc, trial_tones_6ced, trial_tones_188a,
                    trial_tones_1620, trial_tones_e12e, trial_tones_6b0e]
trial_behaviour_expt = [trial_behaviour_5b34, trial_behaviour_d619, trial_behaviour_c2e0, trial_behaviour_735a, 
                        trial_behaviour_1304, trial_behaviour_8de3, trial_behaviour_6b7f, trial_behaviour_1c3f, 
                        trial_behaviour_1604, trial_behaviour_801d, trial_behaviour_1396, trial_behaviour_54db, 
                        trial_behaviour_viol, trial_behaviour_b4c7, trial_behaviour_a12e, trial_behaviour_c653,
                        trial_behaviour_4b7f, trial_behaviour_fc3a, trial_behaviour_0d04, trial_behaviour_1570,
                        trial_behaviour_2099, trial_behaviour_592d, trial_behaviour_a45d, trial_behaviour_0cf1, 
                        trial_behaviour_e453, trial_behaviour_214b, trial_behaviour_e045, trial_behaviour_b7cc, 
                        trial_behaviour_6ced, trial_behaviour_188a, trial_behaviour_1620, trial_behaviour_e12e, 
                        trial_behaviour_6b0e]
corrans_expt = [corrans_5b34, corrans_d619, corrans_c2e0, corrans_735a, corrans_1304, corrans_8de3, corrans_6b7f,
                corrans_1c3f, corrans_1604, corrans_801d, corrans_1396, corrans_54db, corrans_viol, corrans_b4c7,
                corrans_a12e, corrans_c653, corrans_4b7f, corrans_fc3a, corrans_0d04, corrans_1570, corrans_2099, 
                corrans_592d, corrans_a45d, corrans_0cf1, corrans_e453, corrans_214b, corrans_e045, corrans_b7cc, 
                corrans_6ced, corrans_188a, corrans_1620, corrans_e12e, corrans_6b0e]
mleParams_expt = np.zeros((len(trial_tones_expt),6))
mleParams_expt[0,:] = [2.53,2.9,0.055,0.17,0.5,0.46]
mleParams_expt[1,:] = [2.56,2.99,0.051,0.26,0.25,0.46]
mleParams_expt[2,:] = [2.5,2.89,0.047,0.15,0.84,0.51]
mleParams_expt[3,:] = [2.5,2.9,0.056,0.14,0.92,0.5]
mleParams_expt[4,:] = [2.05,3.08,0.048,0.5,0.64,0.62]
mleParams_expt[5,:] = [2.49,2.91,0.052,0.18,0.73,0.5]
mleParams_expt[6,:] = [2.41,3.15,0.051,0.24,0.25,0.26]
mleParams_expt[7,:] = [2.59,2.84,0.052,0.14,0.84,0.52]
mleParams_expt[8,:] = [2.42,3.09,0.05,0.2,0.27,0.26]
mleParams_expt[9,:] = [2.25,3.12,0.05,0.24,0.26,0.66]
mleParams_expt[10,:] = [2.45,2.87,0.055,0.18,0.77,0.65]
mleParams_expt[11,:] = [2.28,3.12,0.054,0.17,0.26,0.51]
mleParams_expt[12,:] = [2.61,2.85,0.048,0.27,0.83,0.48]
mleParams_expt[13,:] = [2.4,3.03,0.05,0.22,0.47,0.58]
mleParams_expt[14,:] = [2.49,2.98,0.065,0.13,0.5,0.28]
mleParams_expt[15,:] = [2.43,2.96,0.051,0.15,0.72,0.5]
mleParams_expt[16,:] = [2.43,3.09,0.04,0.24,0.24,0.3]
mleParams_expt[17,:] = [2.42,3,0.059,0.14,0.9,0.49]
mleParams_expt[18,:] = [2.66,3.01,0.049,0.25,0.46,0.26]
mleParams_expt[19,:] = [2.39,2.95,0.055,0.16,0.69,0.65]
mleParams_expt[20,:] = [2.69,2.71,0.047,0.12,0.83,0.5]
mleParams_expt[21,:] = [2.57,2.98,0.05,0.24,0.7,0.45]
mleParams_expt[22,:] = [2.61,2.83,0.057,0.22,0.23,0.45]
mleParams_expt[23,:] = [2.63,2.89,0.06,0.18,0.5,0.41]
mleParams_expt[24,:] = [2.22,3.14,0.4,0.37,0.79,0.48]
mleParams_expt[25,:] = [2.47,2.95,0.039,0.29,0.82,0.46]
mleParams_expt[26,:] = [2.19,3.39,0.05,0.78,0.5,0.65]
mleParams_expt[27,:] = [2.46,2.9,0.057,0.19,0.26,0.49]
mleParams_expt[28,:] = [2.4,3.02,0.057,0.13,0.77,0.51]
mleParams_expt[29,:] = [2.36,3.06,0.06,0.14,0.43,0.55]
mleParams_expt[30,:] = [2.64,3.16,0.88,0.62,0.89,0.45]
mleParams_expt[31,:] = [2.44,2.94,0.058,0.14,0.63,0.57]
mleParams_expt[32,:] = [3.14,3.42,0.22,0.62,0.92,0.44]

lesionParams_expt = np.zeros((len(trial_tones_expt),6))
lesionParams_expt[0,:] = [2.23,3.3,0.05,0.26,0.0001,0.05]
lesionParams_expt[1,:] = [3.08,3.31,0.048,0.28,0.0001,0.037]
lesionParams_expt[2,:] = [2.1,3.3,0.05,0.25,0.0001,0.85]
lesionParams_expt[3,:] = [2.05,3.27,0.046,0.26,0.0001,0.86]
lesionParams_expt[4,:] = [2.26,3.29,0.66,0.64,0.0001,0.45]
lesionParams_expt[5,:] = [2.04,3.37,0.053,0.21,0.0001,0.47]
lesionParams_expt[6,:] = [2.15,3.36,0.037,0.26,0.0001,0.051]
lesionParams_expt[7,:] = [2.4,3.24,0.051,0.44,0.0001,0.25]
lesionParams_expt[8,:] = [2.15,3.36,0.012,0.25,0.0001,0.65]
lesionParams_expt[9,:] = [2.09,3.33,0.041,0.26,0.0001,0.8]
lesionParams_expt[10,:] = [2.1,3.29,0.05,0.45,0.0001,0.64]
lesionParams_expt[11,:] = [2,3.4,0.052,0.18,0.0001,0.51]
lesionParams_expt[12,:] = [2.63,3.17,0.84,0.67,0.0001,0.46]
lesionParams_expt[13,:] = [2.1,3.33,0.05,0.25,0.0001,0.84]
lesionParams_expt[14,:] = [2.07,3.33,0.055,0.16,0.0001,0.49]
lesionParams_expt[15,:] = [2.09,3.32,0.05,0.25,0.0001,0.47]
lesionParams_expt[16,:] = [1.77,3.64,0.041,0.25,0.0001,0.61]
lesionParams_expt[17,:] = [2,3.33,0.031,0.19,0.0001,0.97]
lesionParams_expt[18,:] = [2.52,3.06,0.053,0.41,0.0001,0.24]
lesionParams_expt[19,:] = [2.1,3.31,0.031,0.26,0.0001,0.84]
lesionParams_expt[20,:] = [2.08,3.33,0.043,0.25,0.0001,0.44]
lesionParams_expt[21,:] = [2.14,3.32,0.052,0.39,0.0001,0.66]
lesionParams_expt[22,:] = [2.27,3.33,0.049,0.26,0.0001,0.048]
lesionParams_expt[23,:] = [2.08,3.33,0.049,0.25,0.0001,0.46]
lesionParams_expt[24,:] = [2.14,3.12,0.054,0.38,0.0001,0.66]
lesionParams_expt[25,:] = [2.82,3.3,0.05,0.46,0.0001,0.05]
lesionParams_expt[26,:] = [2.1,3.04,0.051,0.86,0.0001,0.84]
lesionParams_expt[27,:] = [2.06,3.29,0.048,0.18,0.0001,0.53]
lesionParams_expt[28,:] = [1.96,3.31,0.051,0.18,0.0001,0.98]
lesionParams_expt[29,:] = [1.83,3.51,0.058,0.2,0.0001,0.83]
lesionParams_expt[30,:] = [2.12,2.67,0.46,0.64,0.0001,0.65]
lesionParams_expt[31,:] = [2.08,3.33,0.051,0.25,0.0001,0.84]
lesionParams_expt[32,:] = [2.27,3.33,0.83,0.94,0.0001,0.46]

votingParams_expt = np.zeros((len(trial_tones_expt),2))
votingParams_expt[0,:] = [2.77,0.21]
votingParams_expt[1,:] = [2.72,0.27]
votingParams_expt[2,:] = [2.78,0.23]
votingParams_expt[3,:] = [2.79,0.22]
votingParams_expt[4,:] = [2.64,0.57]
votingParams_expt[5,:] = [2.77,0.2]
votingParams_expt[6,:] = [2.79,0.22]
votingParams_expt[7,:] = [2.77,0.24]
votingParams_expt[8,:] = [2.76,0.23]
votingParams_expt[9,:] = [2.77,0.24]
votingParams_expt[10,:] = [2.72,0.3]
votingParams_expt[11,:] = [2.74,0.17]
votingParams_expt[12,:] = [2.67,0.49]
votingParams_expt[13,:] = [2.75,0.22]
votingParams_expt[14,:] = [2.67,0.15]
votingParams_expt[15,:] = [2.65,0.2]
votingParams_expt[16,:] = [2.74,0.24]
votingParams_expt[17,:] = [2.71,0.16]
votingParams_expt[18,:] = [2.65,0.32]
votingParams_expt[19,:] = [2.77,0.21]
votingParams_expt[20,:] = [2.68,0.21]
votingParams_expt[21,:] = [2.76,0.26]
votingParams_expt[22,:] = [2.78,0.23]
votingParams_expt[23,:] = [2.61,0.25]
votingParams_expt[24,:] = [2.62,0.34]
votingParams_expt[25,:] = [2.65,0.33]
votingParams_expt[26,:] = [3.04,0.75]
votingParams_expt[27,:] = [2.63,0.2]
votingParams_expt[28,:] = [2.64,0.14]
votingParams_expt[29,:] = [2.72,0.14]
votingParams_expt[30,:] = [2.67,0.51]
votingParams_expt[31,:] = [2.72,0.16]
votingParams_expt[32,:] = [2.63,0.83]

allMeanTone_behvSubjArray = np.zeros((len(trial_behaviour_expt),4,len(logFreqBins)))
GaussMeanTone_behvSubjArray = np.zeros((len(trial_behaviour_expt),4,len(logFreqBins)))
negll_fullBayes = np.zeros((len(mleParams_expt),4))
negll_lesionBayes = np.zeros((len(lesionParams_expt),4))
negll_votingModel = np.zeros((len(votingParams_expt),4))
#accuracyAcrossFreq = np.zeros((len(trial_behaviour_expt),8))
noDistractorAccuracy = np.zeros((len(mleParams_expt),))
oneDistractorAccuracy = np.zeros((len(mleParams_expt),))
twoDistractorsAccuracy = np.zeros((len(mleParams_expt),))
threeDistractorsAccuracy = np.zeros((len(mleParams_expt),))
overallAccuracy = np.zeros((len(mleParams_expt),))
predictionAccuracyBayes = np.zeros((len(mleParams_expt),3))
predictionAccuracyLesion = np.zeros((len(mleParams_expt),3))
predictionAccuracyVoting = np.zeros((len(mleParams_expt),3))
AllTonesGaussianAccuracy = np.zeros((len(mleParams_expt),))
OneDistGaussianAccuracy = np.zeros((len(mleParams_expt),))
OneDistGaussianMeanExplained = np.zeros((len(mleParams_expt),))
OneDistGaussianSignalMeanExplained = np.zeros((len(mleParams_expt),))
OneDistOppSidebandLHAccuracy = np.zeros((len(mleParams_expt),))
OneDistOppSidebandLHMeanExplained = np.zeros((len(mleParams_expt),))
OneDistOppSidebandLHSignalMeanExplained = np.zeros((len(mleParams_expt),))
OneDistOppSidebandHLAccuracy = np.zeros((len(mleParams_expt),))
OneDistOppSidebandHLMeanExplained = np.zeros((len(mleParams_expt),))
OneDistOppSidebandHLSignalMeanExplained = np.zeros((len(mleParams_expt),))
TwoDistOppSidebandAccuracy = np.zeros((len(mleParams_expt),)) 
TwoDistOppSidebandMeanExplained = np.zeros((len(mleParams_expt),))
BiasWithUnclearSigLowDist = np.zeros((len(mleParams_expt),2))
BiasWithUnclearSigHighDist = np.zeros((len(mleParams_expt),2))
allGaussLowAccuracy = np.zeros((len(mleParams_expt),))
allGaussHighAccuracy = np.zeros((len(mleParams_expt),))
distGaussLowAccuracy = np.zeros((len(mleParams_expt),))
distGaussHighAccuracy = np.zeros((len(mleParams_expt),))
accuracyVsIdealMeanBhv = np.zeros((len(mleParams_expt),))
accuracyVsIdealSignalMeanBhv = np.zeros((len(mleParams_expt),))
accuracyVsIdealVotingBhv = np.zeros((len(mleParams_expt),))

for iSubj in range(len(trial_behaviour_expt)):
    [idealSubjMeanBhv, 
     idealSubjSignalMeanBhv, 
     idealSubjVotingBhv] = idealSubjectBehaviour(tonesPresented=np.log10(trial_tones_expt[iSubj]))
     
    [accuracyVsIdealMeanBhv[iSubj],
     accuracyVsIdealSignalMeanBhv[iSubj],
     accuracyVsIdealVotingBhv[iSubj]] = compareBehaviourWithDistractors(tonesPresented=np.log10(trial_tones_expt[iSubj]),
                                                                subjectBehaviour=trial_behaviour_expt[iSubj],
                                                                idealMeanBehaviour=idealSubjMeanBhv,
                                                                idealSignalMeanBehaviour=idealSubjSignalMeanBhv,
                                                                idealVotingBehaviour=idealSubjVotingBhv)
    
    [AllGaussLow, AllGaussHigh, DistGaussLow, DistGaussHigh, 
     DistLowOtherLow, DistHighOtherHigh, DistLowOtherHigh, 
     DistHighOtherLow, DistLowOtherLowHigh, DistHighOtherLowHigh,
     DistTwoExtrOtherGauss, DistTwoExtrSameGauss, 
     DistThreeExtr, DistThreeExtrTwoSame, 
     OneDistractor, TwoDistractors] = trialCategories(np.log10(trial_tones_expt[iSubj]), 
                                                      trial_behaviour_expt[iSubj], 
                                                      corrans_expt[iSubj])
    
    Accuracy, BiasWithUnclearSig = accuracies(AllGaussLow, AllGaussHigh, DistGaussLow, DistGaussHigh, 
                                               DistLowOtherLow, DistHighOtherHigh, DistLowOtherHigh, 
                                               DistHighOtherLow, DistLowOtherLowHigh, DistHighOtherLowHigh,
                                               DistTwoExtrOtherGauss, DistTwoExtrSameGauss, 
                                               DistThreeExtrTwoSame, corrans_expt[iSubj], trial_behaviour_expt[iSubj])   
    # Note: accuracy has been multiplied by 100 so is in percentage
    allGaussLowAccuracy[iSubj] = np.mean(AllGaussLow[:,3]==AllGaussLow[:,4])
    allGaussHighAccuracy[iSubj] = np.mean(AllGaussHigh[:,3]==AllGaussHigh[:,4])
    distGaussLowAccuracy[iSubj] = np.mean(DistGaussLow[:,3]==DistGaussLow[:,4])
    distGaussHighAccuracy[iSubj] = np.mean(DistGaussHigh[:,3]==DistGaussHigh[:,4])
    AllTonesGaussianAccuracy[iSubj] = Accuracy[0,0]
    OneDistGaussianAccuracy[iSubj] = Accuracy[2,0]
    OneDistGaussianMeanExplained[iSubj] = Accuracy[2,2]
    OneDistGaussianSignalMeanExplained[iSubj] = Accuracy[2,4]
    OneDistOppSidebandLHAccuracy[iSubj] = Accuracy[4,0]
    OneDistOppSidebandLHMeanExplained[iSubj] = Accuracy[4,2]
    OneDistOppSidebandLHSignalMeanExplained[iSubj] = Accuracy[4,4]
    OneDistOppSidebandHLAccuracy[iSubj] = Accuracy[5,0]
    OneDistOppSidebandHLMeanExplained[iSubj] = Accuracy[5,2]
    OneDistOppSidebandHLSignalMeanExplained[iSubj] = Accuracy[5,4]
    TwoDistOppSidebandAccuracy[iSubj] = Accuracy[6,0]
    TwoDistOppSidebandMeanExplained[iSubj] = Accuracy[6,2]
    BiasWithUnclearSigLowDist[iSubj,:] = BiasWithUnclearSig[:,:2]
    BiasWithUnclearSigHighDist[iSubj,:] = BiasWithUnclearSig[:,2:]
    
    DistractorGaussian = np.concatenate((DistGaussLow,DistGaussHigh),axis=0)
    AllGaussian = np.concatenate((AllGaussLow, AllGaussHigh),axis=0)
    DistractorOppSideband = np.concatenate((DistLowOtherHigh,DistHighOtherLow))
    DistractorSameSideband = np.concatenate((DistLowOtherLow,DistHighOtherHigh))
    ThreeDistractors = np.concatenate((DistThreeExtr,DistThreeExtrTwoSame),axis=0)
    
    noDistractorAccuracy[iSubj] = np.mean(AllGaussian[:,3]==AllGaussian[:,4])
    oneDistractorAccuracy[iSubj] = np.mean(OneDistractor[:,3]==OneDistractor[:,4])
    twoDistractorsAccuracy[iSubj] = np.mean(TwoDistractors[:,3]==TwoDistractors[:,4])
    threeDistractorsAccuracy[iSubj] = np.mean(ThreeDistractors[:,3]==ThreeDistractors[:,4])
    overallAccuracy[iSubj] = np.mean(trial_behaviour_expt[iSubj]==corrans_expt[iSubj])
    
    categoriesForLikelihoodPlot = [AllGaussian, DistractorGaussian, DistractorOppSideband, DistTwoExtrOtherGauss]
    
    #negll_full = MLE(mleParams_expt[iSubj,:], 
    #                 trial_tones=trial_tones_expt[iSubj], 
    #                 trial_behaviour=trial_behaviour_expt[iSubj])
    #negll_lesion = MLE(lesionParams_expt[iSubj,:],
    #                  trial_tones=trial_tones_expt[iSubj], 
    #                  trial_behaviour=trial_behaviour_expt[iSubj])
    #negll_voting = MLE_voting(votingParams_expt[iSubj,:],
    #                  trial_tones=trial_tones_expt[iSubj], 
    #                  trial_behaviour=trial_behaviour_expt[iSubj])

    #for iTrialCategory in range(len(categoriesForLikelihoodPlot)):
    #    negll_fullBayes[iSubj,iTrialCategory] = np.sum(negll_full[categoriesForLikelihoodPlot[iTrialCategory][:,12].astype(int)])
    #    negll_lesionBayes[iSubj,iTrialCategory] = np.sum(negll_lesion[categoriesForLikelihoodPlot[iTrialCategory][:,12].astype(int)])
    #    negll_votingModel[iSubj,iTrialCategory] = np.sum(negll_voting[categoriesForLikelihoodPlot[iTrialCategory][:,12].astype(int)])        
        
    #print('-ve logLikelihood full Bayes different categories',mleParams_expt[iSubj,:],negll_fullBayes[iSubj,:])
    #print('-ve logLikelihood lesion model different categories',lesionParams_expt[iSubj,:],negll_lesionBayes[iSubj,:])
    #print('-ve logLikelihood voting model different categories',votingParams_expt[iSubj,:],negll_votingModel[iSubj,:])
    
    #all_trial_tones, all_trial_behaviour = generate_behaviour(trial_tones_expt[iSubj], 
    #                                                          prob_back=mleParams_expt[iSubj,4], 
    #                                                          prob_low=mleParams_expt[iSubj,5], 
    #                                                          log_prior_params=mleParams_expt[iSubj,:3], 
    #                                                          sigma_sensory=mleParams_expt[iSubj,3], 
    #                                                          reps=1, n_tones=3)
    
    #predictionAccuracyBayes[iSubj,0] = np.mean([np.mean(trial_behaviour_expt[iSubj][allGaussLow[:,12].astype(int)]==all_trial_behaviour[allGaussLow[:,12].astype(int)]),
    #              np.mean(trial_behaviour_expt[iSubj][allGaussHigh[:,12].astype(int)]==all_trial_behaviour[allGaussHigh[:,12].astype(int)]),
    #              np.mean(trial_behaviour_expt[iSubj][distLowOtherLow[:,12].astype(int)]==all_trial_behaviour[distLowOtherLow[:,12].astype(int)]),
    #              np.mean(trial_behaviour_expt[iSubj][distHighOtherHigh[:,12].astype(int)]==all_trial_behaviour[distHighOtherHigh[:,12].astype(int)]),
    #              np.mean(trial_behaviour_expt[iSubj][distTwoExtrSameGauss[:,12].astype(int)]==all_trial_behaviour[distTwoExtrSameGauss[:,12].astype(int)])])
    #predictionAccuracyBayes[iSubj,1] = np.mean([np.mean(trial_behaviour_expt[iSubj][distLowOtherHigh[:,12].astype(int)]==all_trial_behaviour[distLowOtherHigh[:,12].astype(int)]),
    #                  np.mean(trial_behaviour_expt[iSubj][distHighOtherLow[:,12].astype(int)]==all_trial_behaviour[distHighOtherLow[:,12].astype(int)]),
    #                  np.mean(trial_behaviour_expt[iSubj][distGaussHigh[:,12].astype(int)]==all_trial_behaviour[distGaussHigh[:,12].astype(int)]),
    #                  np.mean(trial_behaviour_expt[iSubj][distGaussLow[:,12].astype(int)]==all_trial_behaviour[distGaussLow[:,12].astype(int)])])
    #predictionAccuracyBayes[iSubj,2] = np.mean(trial_behaviour_expt[iSubj][distTwoExtrOtherGauss[:,12].astype(int)]==all_trial_behaviour[distTwoExtrOtherGauss[:,12].astype(int)])

    #allMeanTone_behvSubjArray[iSubj,:,:], GaussMeanTone_behvSubjArray[iSubj,:,:] = plotting(allGaussian, 
    #                                                                                        distractorGaussian, 
    #                                                                                        distLowOtherHigh, 
    #                                                                                        distHighOtherLow)
    

In [ ]:
"""
This computes the last row of the tables, where we are interested in the bias due to all the strategies
as applicable to trials with one tone from gaussians and one tone as distractor.
"""
print((BiasWithUnclearSigLowDist[:,0]+BiasWithUnclearSigHighDist[:,1])/2) 
print((BiasWithUnclearSigLowDist[:,1]+BiasWithUnclearSigHighDist[:,0])/2)
print((BiasWithUnclearSigLowDist[:,1]+BiasWithUnclearSigHighDist[:,0])/
      (BiasWithUnclearSigLowDist[:,0]+BiasWithUnclearSigHighDist[:,1]))

In [ ]:
"""
Analyzing long context data biased towards low
"""

trial_tones_exptLc = [trial_tones_d619Lc, trial_tones_c2e0Lc, trial_tones_735aLc,
                      trial_tones_1304Lc, trial_tones_6b7fLc, trial_tones_1c3fLc,
                      trial_tones_1604Lc, trial_tones_801dLc, trial_tones_1396Lc, 
                      trial_tones_b4c7Lc, trial_tones_a12eLc, trial_tones_c653Lc, 
                      trial_tones_4b7fLc, trial_tones_fc3aLc, trial_tones_0d04Lc,
                      trial_tones_1570Lc, trial_tones_2099Lc, trial_tones_592dLc,
                      trial_tones_a45dLc, trial_tones_0cf1Lc, trial_tones_e453Lc, 
                      trial_tones_214bLc,trial_tones_b7ccLc, trial_tones_6cedLc, 
                      trial_tones_188aLc, trial_tones_e12eLc]
trial_behaviour_exptLc = [trial_behaviour_d619Lc, trial_behaviour_c2e0Lc, trial_behaviour_735aLc,
                          trial_behaviour_1304Lc, trial_behaviour_6b7fLc, trial_behaviour_1c3fLc, 
                          trial_behaviour_1604Lc, trial_behaviour_801dLc, trial_behaviour_1396Lc, 
                          trial_behaviour_b4c7Lc, trial_behaviour_a12eLc, trial_behaviour_c653Lc, 
                          trial_behaviour_4b7fLc, trial_behaviour_fc3aLc, trial_behaviour_0d04Lc,
                          trial_behaviour_1570Lc, trial_behaviour_2099Lc, trial_behaviour_592dLc, 
                          trial_behaviour_a45dLc,
                          trial_behaviour_0cf1Lc, trial_behaviour_e453Lc, trial_behaviour_214bLc,
                          trial_behaviour_b7ccLc, trial_behaviour_6cedLc, trial_behaviour_188aLc,
                          trial_behaviour_e12eLc]
trial_tone_cat_exptLc = [trial_tone_cat_d619Lc, trial_tone_cat_c2e0Lc, trial_tone_cat_735aLc,
                          trial_tone_cat_1304Lc, trial_tone_cat_6b7fLc, trial_tone_cat_1c3fLc, 
                          trial_tone_cat_1604Lc, trial_tone_cat_801dLc, trial_tone_cat_1396Lc, 
                          trial_tone_cat_b4c7Lc, trial_tone_cat_a12eLc, trial_tone_cat_c653Lc, 
                          trial_tone_cat_4b7fLc, trial_tone_cat_fc3aLc, trial_tone_cat_0d04Lc,
                          trial_tone_cat_1570Lc,
                          trial_tone_cat_2099Lc, trial_tone_cat_592dLc, trial_tone_cat_a45dLc,
                          trial_tone_cat_0cf1Lc, trial_tone_cat_e453Lc, trial_tone_cat_214bLc,
                          trial_tone_cat_b7ccLc, trial_tone_cat_6cedLc, trial_tone_cat_188aLc,
                          trial_tone_cat_e12eLc]
corrans_exptLc = [corrans_d619Lc, corrans_c2e0Lc, corrans_735aLc, corrans_1304Lc, 
                  corrans_6b7fLc, corrans_1c3fLc, corrans_1604Lc, corrans_801dLc, 
                  corrans_1396Lc, corrans_b4c7Lc, corrans_a12eLc, corrans_c653Lc, 
                  corrans_4b7fLc, corrans_fc3aLc, corrans_0d04Lc, corrans_1570Lc, corrans_2099Lc, 
                  corrans_592dLc, corrans_a45dLc, corrans_0cf1Lc, corrans_e453Lc, 
                  corrans_214bLc, corrans_b7ccLc, corrans_6cedLc, corrans_188aLc,
                  corrans_e12eLc]

mleParams_exptLc = np.zeros((len(trial_tones_exptLc),6))
mleParams_exptLc[0,:] = [2.53,2.87,0.05,0.19,0.5,0.68]
mleParams_exptLc[1,:] = [2.55,2.82,0.05,0.15,0.51,0.75]
mleParams_exptLc[2,:] = [2.54,2.84,0.06,0.15,0.69,0.61]
mleParams_exptLc[3,:] = [2.54,3.02,0.49,0.83,0.44,0.66]
mleParams_exptLc[4,:] = [2.35,3.12,0.05,0.14,0.26,0.55]
mleParams_exptLc[5,:] = [2.53,2.88,0.05,0.1,0.58,0.72]
mleParams_exptLc[6,:] = [2.56,3,0.05,0.25,0.26,0.45]
mleParams_exptLc[7,:] = [2.64,2.96,0.04,0.37,0.49,0.54]
mleParams_exptLc[8,:] = [2.51,2.89,0.06,0.17,0.68,0.59]
mleParams_exptLc[9,:] = [2.24,3.19,0.06,0.2,0.37,0.61]
mleParams_exptLc[10,:] = [2.39,3.07,0.06,0.14,0.51,0.5]
#mleParams_exptLc[11,:] = [2.39,3.07,0.06,0.14,0.51,0.5]
#mleParams_exptLc[12,:] = [2.39,3.07,0.06,0.14,0.51,0.5]
#mleParams_exptLc[13,:] = [2.39,3.07,0.06,0.14,0.51,0.5]
mleParams_exptLc[14,:] = [2.59,2.82,0.05,0.24,0.86,0.57]
mleParams_exptLc[15,:] = [2.59,2.75,0.048,0.06,0.26,0.75]

allMeanTone_behvSubjArrayLc = np.zeros((len(trial_behaviour_exptLc),4,len(logFreqBins)))
GaussMeanTone_behvSubjArrayLc = np.zeros((len(trial_behaviour_exptLc),4,len(logFreqBins)))
noDistractorAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
oneDistractorAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
twoDistractorsAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
threeDistractorsAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
AllTonesGaussianAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
overallAccuracyLc = np.zeros((len(mleParams_expt),))
OneDistGaussianAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistGaussianMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistGaussianSignalMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistOppSidebandLHAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistOppSidebandLHMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistOppSidebandLHSignalMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistOppSidebandHLAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistOppSidebandHLMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
OneDistOppSidebandHLSignalMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
TwoDistOppSidebandAccuracyLc = np.zeros((len(trial_behaviour_exptLc),)) 
TwoDistOppSidebandMeanExplainedLc = np.zeros((len(trial_behaviour_exptLc),))
allGaussLowAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
allGaussHighAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
distGaussLowAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
distGaussHighAccuracyLc = np.zeros((len(trial_behaviour_exptLc),))
BiasWithUnclearSigLowDistLc = np.zeros((len(mleParams_exptLc),2))
BiasWithUnclearSigHighDistLc = np.zeros((len(mleParams_exptLc),2))

for iSubj in range(len(trial_behaviour_exptLc)):
    [AllGaussLowLc, AllGaussHighLc, DistGaussLowLc, DistGaussHighLc, DistLowOtherLowLc, 
     DistHighOtherHighLc, DistLowOtherHighLc, DistHighOtherLowLc, DistLowOtherLowHighLc, 
     DistHighOtherLowHighLc, DistTwoExtrOtherGaussLc, 
     DistTwoExtrSameGaussLc, DistThreeExtrLc, DistThreeExtrTwoSameLc,
     OneDistractorLc, TwoDistractorsLc] = trialCategories(np.log10(trial_tones_exptLc[iSubj]),
                                                          trial_behaviour_exptLc[iSubj],
                                                          corrans_exptLc[iSubj])
    
    AccuracyLc, BiasWithUnclearSigLc = accuracies(AllGaussLowLc, AllGaussHighLc, DistGaussLowLc, DistGaussHighLc, 
                                                DistLowOtherLowLc, DistHighOtherHighLc, DistLowOtherHighLc, 
                                                DistHighOtherLowLc, DistLowOtherLowHighLc, DistHighOtherLowHighLc,
                                                DistTwoExtrOtherGaussLc, DistTwoExtrSameGaussLc, 
                                                DistThreeExtrTwoSameLc, corrans_exptLc[iSubj], 
                                                trial_behaviour_exptLc[iSubj]) 
    
    # Note: accuracy has been multiplied by 100 so is in percentage
        # Note: accuracy has been multiplied by 100 so is in percentage
    allGaussLowAccuracyLc[iSubj] = np.mean(AllGaussLowLc[:,3]==AllGaussLowLc[:,4])
    allGaussHighAccuracyLc[iSubj] = np.mean(AllGaussHighLc[:,3]==AllGaussHighLc[:,4])
    distGaussLowAccuracyLc[iSubj] = np.mean(DistGaussLowLc[:,3]==DistGaussLowLc[:,4])
    distGaussHighAccuracyLc[iSubj] = np.mean(DistGaussHighLc[:,3]==DistGaussHighLc[:,4])
    AllTonesGaussianAccuracyLc[iSubj] = AccuracyLc[0,0]
    OneDistGaussianAccuracyLc[iSubj] = AccuracyLc[2,0]
    OneDistGaussianMeanExplainedLc[iSubj] = AccuracyLc[2,2]
    OneDistGaussianSignalMeanExplainedLc[iSubj] = AccuracyLc[2,4]
    OneDistOppSidebandLHAccuracyLc[iSubj] = AccuracyLc[4,0]
    OneDistOppSidebandLHMeanExplainedLc[iSubj] = AccuracyLc[4,2]
    OneDistOppSidebandLHSignalMeanExplainedLc[iSubj] = AccuracyLc[4,4]
    OneDistOppSidebandHLAccuracyLc[iSubj] = AccuracyLc[5,0]
    OneDistOppSidebandHLMeanExplainedLc[iSubj] = AccuracyLc[5,2]
    OneDistOppSidebandHLSignalMeanExplainedLc[iSubj] = AccuracyLc[5,4]
    TwoDistOppSidebandAccuracyLc[iSubj] = AccuracyLc[6,0]
    TwoDistOppSidebandMeanExplainedLc[iSubj] = AccuracyLc[6,2]
    BiasWithUnclearSigLowDistLc[iSubj,:] = BiasWithUnclearSigLc[:,:2]
    BiasWithUnclearSigHighDistLc[iSubj,:] = BiasWithUnclearSigLc[:,2:]
    
    DistractorGaussianLc = np.concatenate((DistGaussLowLc,DistGaussHighLc),axis=0)
    AllGaussianLc = np.concatenate((AllGaussLowLc, AllGaussHighLc),axis=0)
    DistractorOppSidebandLc = np.concatenate((DistLowOtherHighLc,DistHighOtherLowLc))
    DistractorSameSidebandLc = np.concatenate((DistLowOtherLowLc,DistHighOtherHighLc))
    ThreeDistractorsLc = np.concatenate((DistThreeExtrLc,DistThreeExtrTwoSameLc),axis=0)
    
    noDistractorAccuracyLc[iSubj] = np.mean(AllGaussianLc[:,3]==AllGaussianLc[:,4])
    oneDistractorAccuracyLc[iSubj] = np.mean(OneDistractorLc[:,3]==OneDistractorLc[:,4])
    twoDistractorsAccuracyLc[iSubj] = np.mean(TwoDistractorsLc[:,3]==TwoDistractorsLc[:,4])
    threeDistractorsAccuracyLc[iSubj] = np.mean(ThreeDistractorsLc[:,3]==ThreeDistractorsLc[:,4])
    overallAccuracyLc[iSubj] = np.mean(trial_behaviour_exptLc[iSubj]==corrans_exptLc[iSubj])
    
    #allMeanTone_behvSubjArrayLc[iSubj,:,:], GaussMeanTone_behvSubjArrayLc[iSubj,:,:] = plotting(allGaussianLc, 
    #                                                                                            distractorGaussianLc, 
    #                                                                                            distLowOtherHighLc, 
    #                                                                                            distHighOtherLowLc)
    

In [ ]:
"""
This computes the last row of the tables, where we are interested in the bias due to all the strategies
as applicable to trials with one tone from gaussians and one tone as distractor.
"""
print((BiasWithUnclearSigLowDistLc[:,0]+BiasWithUnclearSigHighDistLc[:,1])/2) 
print((BiasWithUnclearSigLowDistLc[:,1]+BiasWithUnclearSigHighDistLc[:,0])/2)
print((BiasWithUnclearSigLowDistLc[:,1]+BiasWithUnclearSigHighDistLc[:,0])*100/
      (BiasWithUnclearSigLowDistLc[:,0]+BiasWithUnclearSigHighDistLc[:,1]))

In [ ]:
"""
Analyzing long context data biased towards high
"""

trial_tones_exptHc = [trial_tones_735aHc,
                      trial_tones_1304Hc, trial_tones_6b7fHc, trial_tones_1c3fHc,
                      trial_tones_1604Hc, trial_tones_1396Hc, 
                      trial_tones_b4c7Hc, trial_tones_a12eHc, trial_tones_c653Hc, 
                      trial_tones_4b7fHc, trial_tones_fc3aHc, trial_tones_0d04Hc,
                      trial_tones_2099Hc, trial_tones_592dHc,
                      trial_tones_a45dHc, trial_tones_e453Hc, 
                      trial_tones_214bHc,trial_tones_b7ccHc, trial_tones_6cedHc, 
                      trial_tones_188aHc, trial_tones_e12eHc]
trial_behaviour_exptHc = [trial_behaviour_735aHc,
                          trial_behaviour_1304Hc, trial_behaviour_6b7fHc, trial_behaviour_1c3fHc, 
                          trial_behaviour_1604Hc, trial_behaviour_1396Hc, 
                          trial_behaviour_b4c7Hc, trial_behaviour_a12eHc, trial_behaviour_c653Hc, 
                          trial_behaviour_4b7fHc, trial_behaviour_fc3aHc, trial_behaviour_0d04Hc,
                          trial_behaviour_2099Hc, trial_behaviour_592dHc, 
                          trial_behaviour_a45dHc, trial_behaviour_e453Hc, trial_behaviour_214bHc,
                          trial_behaviour_b7ccHc, trial_behaviour_6cedHc, trial_behaviour_188aHc,
                          trial_behaviour_e12eHc]
trial_tone_cat_exptHc = [trial_tone_cat_735aHc,
                          trial_tone_cat_1304Hc, trial_tone_cat_6b7fHc, trial_tone_cat_1c3fHc, 
                          trial_tone_cat_1604Hc, trial_tone_cat_1396Hc, 
                          trial_tone_cat_b4c7Hc, trial_tone_cat_a12eHc, trial_tone_cat_c653Hc, 
                          trial_tone_cat_4b7fHc, trial_tone_cat_fc3aHc, trial_tone_cat_0d04Hc,
                          trial_tone_cat_2099Hc, trial_tone_cat_592dHc, trial_tone_cat_a45dHc,
                          trial_tone_cat_e453Hc, trial_tone_cat_214bHc,
                          trial_tone_cat_b7ccHc, trial_tone_cat_6cedHc, trial_tone_cat_188aHc,
                          trial_tone_cat_e12eHc]
corrans_exptHc = [corrans_735aHc, corrans_1304Hc, 
                  corrans_6b7fHc, corrans_1c3fHc, corrans_1604Hc, 
                  corrans_1396Hc, corrans_b4c7Hc, corrans_a12eHc, corrans_c653Hc, 
                  corrans_4b7fHc, corrans_fc3aHc, corrans_0d04Hc, corrans_2099Hc, 
                  corrans_592dHc, corrans_a45dHc, corrans_e453Hc, 
                  corrans_214bHc, corrans_b7ccHc, corrans_6cedHc, corrans_188aHc,
                  corrans_e12eHc]

mleParams_exptHc = np.zeros((len(trial_tones_exptHc),6))

allMeanTone_behvSubjArrayHc = np.zeros((len(trial_behaviour_exptHc),4,len(logFreqBins)))
GaussMeanTone_behvSubjArrayHc = np.zeros((len(trial_behaviour_exptHc),4,len(logFreqBins)))
noDistractorAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
oneDistractorAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
twoDistractorsAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
threeDistractorsAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
AllTonesGaussianAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
overallAccuracyHc = np.zeros((len(mleParams_exptHc),))
OneDistGaussianAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistGaussianMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistGaussianSignalMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistOppSidebandLHAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistOppSidebandLHMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistOppSidebandLHSignalMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistOppSidebandHLAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistOppSidebandHLMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
OneDistOppSidebandHLSignalMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
TwoDistOppSidebandAccuracyHc = np.zeros((len(trial_behaviour_exptHc),)) 
TwoDistOppSidebandMeanExplainedHc = np.zeros((len(trial_behaviour_exptHc),))
allGaussLowAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
allGaussHighAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
distGaussLowAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
distGaussHighAccuracyHc = np.zeros((len(trial_behaviour_exptHc),))
BiasWithUnclearSigLowDistHc = np.zeros((len(mleParams_exptHc),2))
BiasWithUnclearSigHighDistHc = np.zeros((len(mleParams_exptHc),2))

for iSubj in [15,16]:#range(len(trial_behaviour_exptHc)):
    [AllGaussLowHc, AllGaussHighHc, DistGaussLowHc, DistGaussHighHc, DistLowOtherLowHc, 
     DistHighOtherHighHc, DistLowOtherHighHc, DistHighOtherLowHc, DistLowOtherLowHighHc, 
     DistHighOtherLowHighHc, DistTwoExtrOtherGaussHc, 
     DistTwoExtrSameGaussHc, DistThreeExtrHc, DistThreeExtrTwoSameHc,
     OneDistractorHc, TwoDistractorsHc] = trialCategories(np.log10(trial_tones_exptHc[iSubj]),
                                                          trial_behaviour_exptHc[iSubj],
                                                          corrans_exptHc[iSubj])
    
    AccuracyHc, BiasWithUnclearSigHc = accuracies(AllGaussLowHc, AllGaussHighHc, DistGaussLowHc, DistGaussHighHc, 
                                                DistLowOtherLowHc, DistHighOtherHighHc, DistLowOtherHighHc, 
                                                DistHighOtherLowHc, DistLowOtherLowHighHc, DistHighOtherLowHighHc,
                                                DistTwoExtrOtherGaussHc, DistTwoExtrSameGaussHc, 
                                                DistThreeExtrTwoSameHc, corrans_exptHc[iSubj], 
                                                trial_behaviour_exptHc[iSubj]) 
    
    # Note: accuracy has been multiplied by 100 so is in percentage
        # Note: accuracy has been multiplied by 100 so is in percentage
    allGaussLowAccuracyHc[iSubj] = np.mean(AllGaussLowHc[:,3]==AllGaussLowHc[:,4])
    allGaussHighAccuracyHc[iSubj] = np.mean(AllGaussHighHc[:,3]==AllGaussHighHc[:,4])
    distGaussLowAccuracyHc[iSubj] = np.mean(DistGaussLowHc[:,3]==DistGaussLowHc[:,4])
    distGaussHighAccuracyHc[iSubj] = np.mean(DistGaussHighHc[:,3]==DistGaussHighHc[:,4])
    AllTonesGaussianAccuracyHc[iSubj] = AccuracyHc[0,0]
    OneDistGaussianAccuracyHc[iSubj] = AccuracyHc[2,0]
    OneDistGaussianMeanExplainedHc[iSubj] = AccuracyHc[2,2]
    OneDistGaussianSignalMeanExplainedHc[iSubj] = AccuracyHc[2,4]
    OneDistOppSidebandLHAccuracyHc[iSubj] = AccuracyHc[4,0]
    OneDistOppSidebandLHMeanExplainedHc[iSubj] = AccuracyHc[4,2]
    OneDistOppSidebandLHSignalMeanExplainedHc[iSubj] = AccuracyHc[4,4]
    OneDistOppSidebandHLAccuracyHc[iSubj] = AccuracyHc[5,0]
    OneDistOppSidebandHLMeanExplainedHc[iSubj] = AccuracyHc[5,2]
    OneDistOppSidebandHLSignalMeanExplainedHc[iSubj] = AccuracyHc[5,4]
    TwoDistOppSidebandAccuracyHc[iSubj] = AccuracyHc[6,0]
    TwoDistOppSidebandMeanExplainedHc[iSubj] = AccuracyHc[6,2]
    BiasWithUnclearSigLowDistHc[iSubj,:] = BiasWithUnclearSigHc[:,:2]
    BiasWithUnclearSigHighDistHc[iSubj,:] = BiasWithUnclearSigHc[:,2:]
    
    DistractorGaussianHc = np.concatenate((DistGaussLowHc,DistGaussHighHc),axis=0)
    AllGaussianHc = np.concatenate((AllGaussLowHc, AllGaussHighHc),axis=0)
    DistractorOppSidebandHc = np.concatenate((DistLowOtherHighHc,DistHighOtherLowHc))
    DistractorSameSidebandHc = np.concatenate((DistLowOtherLowHc,DistHighOtherHighHc))
    ThreeDistractorsHc = np.concatenate((DistThreeExtrHc,DistThreeExtrTwoSameHc),axis=0)
    
    noDistractorAccuracyHc[iSubj] = np.mean(AllGaussianHc[:,3]==AllGaussianHc[:,4])
    oneDistractorAccuracyHc[iSubj] = np.mean(OneDistractorHc[:,3]==OneDistractorHc[:,4])
    twoDistractorsAccuracyHc[iSubj] = np.mean(TwoDistractorsHc[:,3]==TwoDistractorsHc[:,4])
    threeDistractorsAccuracyHc[iSubj] = np.mean(ThreeDistractorsHc[:,3]==ThreeDistractorsHc[:,4])
    overallAccuracyHc[iSubj] = np.mean(trial_behaviour_exptHc[iSubj]==corrans_exptHc[iSubj])
    
    #allMeanTone_behvSubjArrayHc[iSubj,:,:], GaussMeanTone_behvSubjArrayHc[iSubj,:,:] = plotting(allGaussianHc, 
    #                                                                                            distractorGaussianHc, 
    #                                                                                            distLowOtherHighHc, 
    #                                                                                            distHighOtherLowHc)
    

In [ ]:
"""
This computes the last row of the tables, where we are interested in the bias due to all the strategies
as applicable to trials with one tone from gaussians and one tone as distractor.
"""
print((BiasWithUnclearSigLowDistHc[:,0]+BiasWithUnclearSigHighDistHc[:,1])/2) 
print((BiasWithUnclearSigLowDistHc[:,1]+BiasWithUnclearSigHighDistHc[:,0])/2)
print((BiasWithUnclearSigLowDistHc[:,1]+BiasWithUnclearSigHighDistHc[:,0])*100/
      (BiasWithUnclearSigLowDistHc[:,0]+BiasWithUnclearSigHighDistHc[:,1]))

In [ ]:
sig = 0.1
lowMu = 2.55
highMu = 2.85
    
def BehaviourGivenOneBackground(trial_tones_expt,
                                trial_behaviour_expt,
                                trial_tone_cat_expt, 
                                corrans_expt):
    unique_tones = np.unique(trial_tones_expt)

    tone1_prob_behaviour = np.zeros((len(unique_tones)))
    tone2_prob_behaviour = np.zeros((len(unique_tones)))
    tone3_prob_behaviour = np.zeros((len(unique_tones)))

    for i_tone in range(len(unique_tones)):
        tone1_prob_behaviour[i_tone] = np.mean(trial_behaviour_expt[trial_tones_expt[:,0]\
                                                           ==unique_tones[i_tone]])
        tone2_prob_behaviour[i_tone] = np.mean(trial_behaviour_expt[trial_tones_expt[:,1]\
                                                           ==unique_tones[i_tone]])
        tone3_prob_behaviour[i_tone] = np.mean(trial_behaviour_expt[trial_tones_expt[:,2]\
                                                           ==unique_tones[i_tone]])

    #influence1, = plt.plot(np.log10(unique_tones), tone1_prob_behavior, label = 'Influence of Tone 1')
    #influence2, = plt.plot(np.log10(unique_tones), tone2_prob_behavior, label = 'Influence of Tone 2')
    #influence3, = plt.plot(np.log10(unique_tones), tone3_prob_behavior, label = 'Influence of Tone 3')
    #influence, = plt.plot(np.log10(unique_tones), (tone1_prob_behavior+tone2_prob_behavior+tone3_prob_behavior)/3,
                           #'k', label = 'Average Influence')

    influence = (tone1_prob_behaviour+tone2_prob_behaviour+tone3_prob_behaviour)/3
    mnTones = np.mean(trial_tones_expt,1)
    counts, bins = np.histogram(mnTones)
    binID = np.digitize(mnTones,bins)
    un = np.unique(binID)

    mnR = np.zeros(len(un))
    for idx, val in enumerate(un):
        mnR[idx] = np.mean(trial_behaviour_expt[binID == val])

    tempCat = np.copy(trial_tone_cat_expt)

    tempCat[tempCat == 1] = -1 
    tempCat[tempCat == 2] = 1

    sum2 = np.sum(tempCat,1)

    un2 = np.unique(sum2)

    mnR2 = np.zeros(len(un2))
    for idx, val in enumerate(un2):
        mnR2[idx] = np.mean(trial_behaviour_expt[sum2 == val])    
        
    #Let's look only at stimuli where 2 were from one category, and one was a background

    high_1B_idx = sum2 == 2
    low_1B_idx = sum2 == -2

    # For High:

    hTemp = np.copy(trial_tone_cat_expt[high_1B_idx,:])

    temp2 = np.zeros(len(hTemp))
    temp3 = np.zeros(len(hTemp))

    trialTonesTemp = np.copy(trial_tones_expt[high_1B_idx,:])
    behaviorTemp = np.copy(trial_behaviour_expt[high_1B_idx])

    for idx, val in enumerate(hTemp):
        temp = np.where(hTemp[idx,:] == 0)    
        others = np.where(hTemp[idx,:] != 0)    
        temp2[idx] = np.log10(trialTonesTemp[idx,temp])
        t = np.log10(trialTonesTemp[idx,others])
        othersMean = np.mean(t)#(t[:,0]*t[:,1])**(1/2)    
        temp3[idx] = (temp2[idx] - othersMean)

    counts, highBins = np.histogram(temp2)
    binID = np.digitize(temp2,highBins)
    un = np.unique(binID)

    mnRespHigh = np.zeros(len(un))
    for idx, val in enumerate(un):
        mnRespHigh[idx] = np.mean(behaviorTemp[binID == val])


    # For Low:

    lTemp = np.copy(trial_tone_cat_expt[low_1B_idx,:])

    temp2 = np.zeros(len(lTemp))
    temp3 = np.zeros(len(lTemp))

    trialTonesTemp = np.copy(trial_tones_expt[low_1B_idx,:])
    behaviorTemp = np.copy(trial_behaviour_expt[low_1B_idx])

    for idx, val in enumerate(lTemp):
        temp = np.where(lTemp[idx,:] == 0)
        others = np.where(lTemp[idx,:] != 0)
        temp2[idx] = np.log10(trialTonesTemp[idx,temp])   
        t = np.log10(trialTonesTemp[idx,others])
        othersMean = np.mean(t)
        temp3[idx] = (temp2[idx] - othersMean)
    # Now, to Z-score them:

    #zScoreLow = np.copy((np.log10(temp2) - lowMu) / sig)
    zScoreLow = np.copy(temp2)
    #zScoreLow = np.copy(temp3)

    counts, lowBins = np.histogram(zScoreLow)
    binID = np.digitize(zScoreLow,lowBins)
    un = np.unique(binID)

    mnRespLow = np.zeros(len(un))
    for idx, val in enumerate(un):
        mnRespLow[idx] = np.mean(behaviorTemp[binID == val])

    avgRespLow = np.mean(trial_behaviour_expt[sum2 == -3])  
    avgRespHigh = np.mean(trial_behaviour_expt[sum2 == 3])   

    return lowBins, highBins, mnRespLow, mnRespHigh, avgRespLow, avgRespHigh


In [ ]:
[lowBins, highBins, 
 mnRespLow, mnRespHigh, 
 avgRespLow, avgRespHigh] = BehaviourGivenOneBackground(trial_tones_a12e,
                                                        trial_behaviour_a12e,
                                                        trial_tone_cat_a12e,
                                                        corrans_a12e)

[lowBinsLc, highBinsLc, 
 mnRespLowLc, mnRespHighLc, 
 avgRespLowLc, avgRespHighLc] = BehaviourGivenOneBackground(trial_tones_a12eLc,
                                                            trial_behaviour_a12eLc,
                                                            trial_tone_cat_a12eLc,
                                                            corrans_a12eLc)

fig, axs = plt.subplots(2, 1, sharex = True, sharey = True)
axs[0].plot(lowBins,mnRespLow,'b-')
axs[0].plot(lowMu, avgRespLow, 'bo')  
axs[0].plot(lowBinsLc, mnRespLowLc, 'b--')
axs[0].axhline(y=0, color='k', linestyle='--')

axs[1].plot(highBins, mnRespHigh,'r-')
axs[1].plot(highMu, avgRespHigh, 'ro')
axs[1].plot(highBinsLc, mnRespHighLc, 'r--')
axs[1].axhline(y=0, color='k', linestyle='--')

fig = plt.figure()  # create a figure object
ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
ax.plot(lowBins, mnRespLow-mnRespLowLc, 'b--')
ax.plot(highBins, mnRespHigh-mnRespHighLc, 'r--')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel('log10(frequency)')
plt.ylabel('p(B_H)')
ax.legend(['Two tones are low','Two tones are high'])
#plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/ChangeInBehaviourWithBackground_735a')

In [ ]:

def computeAggregate():

    AllTonesGaussianMeanExplained = AllTonesGaussianAccuracy
    TwoDistOppSidebandSignalMeanExplained = 100-TwoDistOppSidebandMeanExplained
    
    FullBayesModel = []
    PbackLesionModel = []
    VotingModel = []
    
    """
    Mean values of model parameters
    """
    print('Complete model')
    print('mu low', np.mean(mleParams_expt[AllTonesGaussianAccuracy>=70,0]),np.std(mleParams_expt[AllTonesGaussianAccuracy>=70,0])/np.sqrt(30))
    print('mu high', np.mean(mleParams_expt[AllTonesGaussianAccuracy>=70,1]),np.std(mleParams_expt[AllTonesGaussianAccuracy>=70,1])/np.sqrt(30))
    print('sigma distribution', np.mean(mleParams_expt[AllTonesGaussianAccuracy>=70,2]),np.std(mleParams_expt[AllTonesGaussianAccuracy>=70,2])/np.sqrt(30))
    print('sigma sensory', np.mean(mleParams_expt[AllTonesGaussianAccuracy>=70,3]),np.std(mleParams_expt[AllTonesGaussianAccuracy>=70,3])/np.sqrt(30))
    print('pback', np.mean(mleParams_expt[AllTonesGaussianAccuracy>=70,4]),np.std(mleParams_expt[AllTonesGaussianAccuracy>=70,4])/np.sqrt(30))
    print('plow', np.mean(mleParams_expt[AllTonesGaussianAccuracy>=70,5]),np.std(mleParams_expt[AllTonesGaussianAccuracy>=70,5])/np.sqrt(30))
    
    print('Lesion model')
    print('mu low', np.mean(lesionParams_expt[AllTonesGaussianAccuracy>=70,0]),np.std(lesionParams_expt[AllTonesGaussianAccuracy>=70,0])/np.sqrt(30))
    print('mu high', np.mean(lesionParams_expt[AllTonesGaussianAccuracy>=70,1]),np.std(lesionParams_expt[AllTonesGaussianAccuracy>=70,1])/np.sqrt(30))
    print('sigma distribution', np.mean(lesionParams_expt[AllTonesGaussianAccuracy>=70,2]),np.std(lesionParams_expt[AllTonesGaussianAccuracy>=70,2])/np.sqrt(30))
    print('sigma sensory', np.mean(lesionParams_expt[AllTonesGaussianAccuracy>=70,3]),np.std(lesionParams_expt[AllTonesGaussianAccuracy>=70,3])/np.sqrt(30))
    print('pback', np.mean(lesionParams_expt[AllTonesGaussianAccuracy>=70,4]),np.std(lesionParams_expt[AllTonesGaussianAccuracy>=70,4])/np.sqrt(30))
    print('plow', np.mean(lesionParams_expt[AllTonesGaussianAccuracy>=70,5]),np.std(lesionParams_expt[AllTonesGaussianAccuracy>=70,5])/np.sqrt(30))
    
    print('Voting model')
    print('decision boundary', np.mean(votingParams_expt[AllTonesGaussianAccuracy>=70,0]), np.std(votingParams_expt[AllTonesGaussianAccuracy>=70,0])/np.sqrt(30))
    print('sigma distribution', np.mean(votingParams_expt[AllTonesGaussianAccuracy>=70,1]), np.std(votingParams_expt[AllTonesGaussianAccuracy>=70,1])/np.sqrt(30))
    
    """
    Accuracy vs % of data explained using strategies for all subjects in the no context case
    """
    
    fig = plt.figure(tight_layout=True)  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.hist(AllTonesGaussianAccuracy, bins=[50,60,70,80,90,100])
    ax.set_ylim(0,30)
    ax.set_xlim(50,100)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Accuracy',fontsize=15)
    plt.ylabel('Number of participants',fontsize=15)
    #plt.savefig('figures/FromProlific/rawDataAnalysis/SubjectsToExclude')
    
    fig = plt.figure(tight_layout=True)  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot([0,1,2,3],[noDistractorAccuracy[AllTonesGaussianAccuracy>=70]*100,
                       oneDistractorAccuracy[AllTonesGaussianAccuracy>=70]*100,
                       twoDistractorsAccuracy[AllTonesGaussianAccuracy>=70]*100,
                       threeDistractorsAccuracy[AllTonesGaussianAccuracy>=70]*100],'k.')
    #ax.errorbar(x=[0,1,2,3],y=[np.median(noDistractorAccuracy[AllTonesGaussianAccuracy>=70])*100,
    #                           np.median(oneDistractorAccuracy[AllTonesGaussianAccuracy>=70])*100,
    #                           np.median(twoDistractorsAccuracy[AllTonesGaussianAccuracy>=70])*100,
    #                           np.median(threeDistractorsAccuracy[AllTonesGaussianAccuracy>=70])*100],
    #            yerr = [np.std(noDistractorAccuracy[AllTonesGaussianAccuracy>=70])/np.sqrt(30)*100,
    #                    np.std(oneDistractorAccuracy[AllTonesGaussianAccuracy>=70])/np.sqrt(30)*100,
    #                    np.std(twoDistractorsAccuracy[AllTonesGaussianAccuracy>=70])/np.sqrt(30)*100,
    #                    np.std(threeDistractorsAccuracy[AllTonesGaussianAccuracy>=70])/np.sqrt(30)*100],
    #            marker='.',color='r',linestyle='')
    ax.set_ylim(0,100)
    ax.set_xlim(-0.5,4.5)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Number of Distractors',fontsize=15)
    plt.ylabel('Accuracy',fontsize=15)
    #plt.savefig('figures/FromProlific/rawDataAnalysis/AccuracyVsNoOfDistractors')    
    
    fig = plt.figure(figsize=(20,10),tight_layout=True)  # create a figure object
    ax = fig.add_subplot(3,5,1)
    ax.plot(AllTonesGaussianAccuracy[AllTonesGaussianAccuracy>=70],
            AllTonesGaussianMeanExplained[AllTonesGaussianAccuracy>=70], 'b.',markersize=12)
    ax.plot(AllTonesGaussianAccuracy[[24]],AllTonesGaussianMeanExplained[[24]],'ks',markersize=12)
    ax.set_ylim(0,100); ax.set_xlim(10,100); plt.xticks([]); plt.yticks(fontsize=20)
    ax2 = fig.add_subplot(3,5,6)
    ax2.plot(AllTonesGaussianAccuracy[AllTonesGaussianAccuracy>=70],
            AllTonesGaussianMeanExplained[AllTonesGaussianAccuracy>=70], 'r.',markersize=12)
    ax2.plot(AllTonesGaussianAccuracy[[3,7,20]],AllTonesGaussianMeanExplained[[3,7,20]],'k*',markersize=12)
    ax2.set_ylim(0,100); ax2.set_xlim(10,100); plt.xticks([]); plt.yticks(fontsize=20)
    ax3 = fig.add_subplot(3,5,11)
    ax3.plot(AllTonesGaussianAccuracy[AllTonesGaussianAccuracy>=70],
            AllTonesGaussianMeanExplained[AllTonesGaussianAccuracy>=70], 'g.',markersize=12)
    ax3.plot(AllTonesGaussianAccuracy[[14,28,29]],AllTonesGaussianMeanExplained[[14,28,29]],'kP',markersize=14)
    ax3.set_ylim(0,100); ax3.set_xlim(10,100); plt.xticks(fontsize=20); plt.yticks(fontsize=20)    
    ax = fig.add_subplot(3,5,2)
    ax.plot(OneDistGaussianAccuracy[AllTonesGaussianAccuracy>=70], 
            OneDistGaussianMeanExplained[AllTonesGaussianAccuracy>=70], 'b.',markersize=12)
    ax.plot(OneDistGaussianAccuracy[[24]],OneDistGaussianMeanExplained[[24]],'ks',markersize=12)
    ax.set_ylim(0,100); ax.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax2 = fig.add_subplot(3,5,7)
    ax2.plot(OneDistGaussianAccuracy[AllTonesGaussianAccuracy>=70], 
            OneDistGaussianSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'r.',markersize=12)
    ax2.plot(OneDistGaussianAccuracy[[3,7,20]],OneDistGaussianSignalMeanExplained[[3,7,20]],'k*',markersize=12)
    ax2.set_ylim(0,100); ax2.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax3 = fig.add_subplot(3,5,12)
    ax3.plot(OneDistGaussianAccuracy[AllTonesGaussianAccuracy>=70], 
            OneDistGaussianSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'g.',markersize=12)
    ax3.plot(OneDistGaussianAccuracy[[14,28,29]],OneDistGaussianSignalMeanExplained[[14,28,29]],'kP',markersize=14)
    ax3.set_ylim(0,100); ax3.set_xlim(10,100); plt.xticks(fontsize=20); plt.yticks([])  
    ax = fig.add_subplot(3,5,3)
    ax.plot(OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70], 
            OneDistOppSidebandLHMeanExplained[AllTonesGaussianAccuracy>=70], 'b.',markersize=12)
    ax.plot(OneDistOppSidebandLHAccuracy[[24]], 
            OneDistOppSidebandLHMeanExplained[[24]], 'ks',markersize=12)
    ax.set_ylim(0,100); ax.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax2 = fig.add_subplot(3,5,8)
    ax2.plot(OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70],
            OneDistOppSidebandLHSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'r.',markersize=12)
    ax2.plot(OneDistOppSidebandLHAccuracy[[3,7,20]], 
            OneDistOppSidebandLHSignalMeanExplained[[3,7,20]], 'k*',markersize=12)
    ax2.set_ylim(0,100); ax2.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax3 = fig.add_subplot(3,5,13)
    ax3.plot(OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70],
            OneDistOppSidebandLHSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'g.',markersize=12)
    ax3.plot(OneDistOppSidebandLHAccuracy[[14,28,29]], 
            OneDistOppSidebandLHSignalMeanExplained[[14,28,29]], 'kP',markersize=14)
    ax3.set_ylim(0,100); ax3.set_xlim(10,100); plt.xticks(fontsize=20); plt.yticks([])    
    ax = fig.add_subplot(3,5,4)
    ax.plot(OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70],
            OneDistOppSidebandHLMeanExplained[AllTonesGaussianAccuracy>=70], 'b.',markersize=12)
    ax.plot(OneDistOppSidebandHLAccuracy[[24]], 
            OneDistOppSidebandHLMeanExplained[[24]], 'ks',markersize=12)
    ax.set_ylim(0,100); ax.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax2 = fig.add_subplot(3,5,9)
    ax2.plot(OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70],
            OneDistOppSidebandHLSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'r.',markersize=12)
    ax2.plot(OneDistOppSidebandHLAccuracy[[3,7,20]], 
            OneDistOppSidebandHLSignalMeanExplained[[3,7,20]], 'k*',markersize=12)
    ax2.set_ylim(0,100); ax2.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax3 = fig.add_subplot(3,5,14)
    ax3.plot(OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70],
            OneDistOppSidebandHLSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'g.',markersize=12)
    ax3.plot(OneDistOppSidebandHLAccuracy[[14,28,29]], 
            OneDistOppSidebandHLSignalMeanExplained[[14,28,29]], 'kP',markersize=14)
    ax3.set_ylim(0,100); ax3.set_xlim(10,100); plt.xticks(fontsize=20); plt.yticks([])    
    ax = fig.add_subplot(3,5,5)
    ax.plot(TwoDistOppSidebandAccuracy[AllTonesGaussianAccuracy>=70],
            TwoDistOppSidebandMeanExplained[AllTonesGaussianAccuracy>=70], 'b.',markersize=12)
    ax.plot(TwoDistOppSidebandAccuracy[[24]], TwoDistOppSidebandMeanExplained[[24]], 'ks',markersize=12)
    ax.set_ylim(0,100); ax.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax2 = fig.add_subplot(3,5,10)
    ax2.plot(TwoDistOppSidebandAccuracy[AllTonesGaussianAccuracy>=70],
            TwoDistOppSidebandSignalMeanExplained[AllTonesGaussianAccuracy>=70], 'r.',markersize=12)
    ax2.plot(TwoDistOppSidebandAccuracy[[3,7,20]], TwoDistOppSidebandSignalMeanExplained[[3,7,20]], 'k*',markersize=12)
    ax2.set_ylim(0,100); ax2.set_xlim(10,100); plt.xticks([]); plt.yticks([])
    ax3 = fig.add_subplot(3,5,15)
    ax3.plot(TwoDistOppSidebandAccuracy[AllTonesGaussianAccuracy>=70],
            TwoDistOppSidebandMeanExplained[AllTonesGaussianAccuracy>=70], 'g.',markersize=12)
    ax3.plot(TwoDistOppSidebandAccuracy[[14,28,29]], TwoDistOppSidebandMeanExplained[[14,28,29]], 'kP',markersize=14)
    ax3.set_ylim(0,100); ax3.set_xlim(0,100); plt.xticks(fontsize=20); plt.yticks([])
    #plt.savefig('figures/FromProlific/rawDataAnalysis/AccuracyAllStrategies') 
    
    fig = plt.figure(figsize=(7,5),tight_layout=True)  # create a figure object
    ax = fig.add_subplot(1,1,1)
    ax.plot((BiasWithUnclearSigLowDist[AllTonesGaussianAccuracy>=70,0]+
             BiasWithUnclearSigHighDist[AllTonesGaussianAccuracy>=70,1])/2, 
            (BiasWithUnclearSigLowDist[AllTonesGaussianAccuracy>=70,1]+
             BiasWithUnclearSigHighDist[AllTonesGaussianAccuracy>=70,0])/2,'b.')
    ax.plot((BiasWithUnclearSigLowDist[[24],0]+BiasWithUnclearSigHighDist[[24],1])/2, 
            (BiasWithUnclearSigLowDist[[24],1]+BiasWithUnclearSigHighDist[[24],0])/2,'ks',markersize=12)
    ax.plot((BiasWithUnclearSigLowDist[[3,7,20],0]+BiasWithUnclearSigHighDist[[3,7,20],1])/2, 
            (BiasWithUnclearSigLowDist[[3,7,20],1]+BiasWithUnclearSigHighDist[[3,7,20],0])/2,'k*',markersize=12)
    ax.plot((BiasWithUnclearSigLowDist[[14,28,29],0]+BiasWithUnclearSigHighDist[[14,28,29],1])/2, 
            (BiasWithUnclearSigLowDist[[14,28,29],1]+BiasWithUnclearSigHighDist[[14,28,29],0])/2,'kP',markersize=14)
    ax.set_xlabel('% of data when distractors lie on same side',fontsize=15)
    ax.set_ylabel('% of data when distractors lie on opposite side',fontsize=15)
    ax.set_ylim(0,50); ax.set_xlim(50,100); plt.xticks(fontsize=15); plt.yticks(fontsize=15)
    #plt.savefig('figures/FromProlific/rawDataAnalysis/BiasWithUnclearSignalData') 
    
    """
    Accuracy vs % of data explained using different models in the no context case
    """
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(mleParams_expt[AllTonesGaussianAccuracy>=70,4],
            (OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70]+
             OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70]+
             TwoDistOppSidebandAccuracy[AllTonesGaussianAccuracy>=70])/3,'k.')
    ax.set_ylim(0,100)
    ax.set_xlim(0,1)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('pback',fontsize=15)
    plt.ylabel('Accuracy',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/pbackVsAccuracy')
    """
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(overallAccuracy[AllTonesGaussianAccuracy>=70]*100, 
            negll_fullBayes[AllTonesGaussianAccuracy>=70,0],'k.')
    ax.set_xlim(0,100)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Accuracy',fontsize=15)
    plt.ylabel('-ve LogLikelihood of Full Bayesian Model',fontsize=15)
    plt.tight_layout()
    plt.savefig('figures/FromProlific/rawDataAnalysis/BayesianModelLikelihoodAgainstAccuracy.png')
    """
        
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,0], 
            negll_lesionBayes[AllTonesGaussianAccuracy>=70,0],color='orange',marker='.',linestyle='None')
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,1],
            negll_lesionBayes[AllTonesGaussianAccuracy>=70,1],color='lightseagreen',marker='.',linestyle='None')
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,2],
            negll_lesionBayes[AllTonesGaussianAccuracy>=70,2],color='red',marker='.',linestyle='None')
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,3],
            negll_lesionBayes[AllTonesGaussianAccuracy>=70,3],color='black',marker='.',linestyle='None')
    ax.plot(np.arange(100),np.arange(100),'k--')
    ax.set_xlim(0,100)
    ax.set_ylim(0,140)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('-ve LogLikelihood of Bayesian Model',fontsize=15)
    plt.ylabel('-ve LogLikelihood of Lesion Model',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/LikelihoodFullVsLesion.png')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,0], 
            negll_votingModel[AllTonesGaussianAccuracy>=70,0],color='orange',marker='.',linestyle='None')
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,1],
            negll_votingModel[AllTonesGaussianAccuracy>=70,1],color='lightseagreen',marker='.',linestyle='None')
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,2],
            negll_votingModel[AllTonesGaussianAccuracy>=70,2],color='red',marker='.',linestyle='None')
    ax.plot(negll_fullBayes[AllTonesGaussianAccuracy>=70,3],
            negll_votingModel[AllTonesGaussianAccuracy>=70,3],color='black',marker='.',linestyle='None')
    ax.plot(np.arange(100),np.arange(100),'k--')
    ax.set_xlim(0,100)
    ax.set_ylim(0,100)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('-ve LogLikelihood of Bayesian Model',fontsize=15)
    plt.ylabel('-ve LogLikelihood of Voting Model',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/LikelihoodFullVsVoting.png')
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1) 
    mycmap=cm.Greys
    ax.scatter((OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70]
                +OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70])/2,
                TwoDistOppSidebandAccuracy[AllTonesGaussianAccuracy>=70],
                c=mycmap(mleParams_expt[AllTonesGaussianAccuracy>=70,4]),s=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Accuracy One Sideband Distractor',fontsize=15)
    plt.ylabel('Accuracy Two Sideband Distractors',fontsize=15)
    plt.tight_layout() 
    cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=mycmap))
    cbar.ax.tick_params(labelsize=15) 
    plt.savefig('figures/FromProlific/rawDataAnalysis/2dColormapPback.png')

    """
    Plotting average p(B_H|mean tone) and average p(B_H|mean of signal tones)
    """
    _, bin_edges = np.histogram((OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70]+
                                 OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70]+
                                 OneDistGaussianAccuracy[AllTonesGaussianAccuracy>=70])/3,bins=4)
    subjectBins = np.digitize((OneDistOppSidebandHLAccuracy[AllTonesGaussianAccuracy>=70]+
                               OneDistOppSidebandLHAccuracy[AllTonesGaussianAccuracy>=70]+
                               OneDistGaussianAccuracy[AllTonesGaussianAccuracy>=70])/3, bin_edges)
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,0,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,0,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,1,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,1,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,2,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,2,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,3,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,3,:],axis=0)/sum(subjectBins==1))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of all 3 tones',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/AllMeanToneProbHighBadSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,0,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,0,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,1,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,1,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,2,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,2,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,3,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,3,:],axis=0)/sum(subjectBins==2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of all 3 tones',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/AllMeanToneProbHighMediocreSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,0,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,0,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,1,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,1,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,2,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,2,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,3,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,3,:],axis=0)/sum(subjectBins>2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of all 3 tones',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/AllMeanToneProbHighGoodSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,0,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,0,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,1,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,1,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,2,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,2,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,3,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==1,3,:],axis=0)/sum(subjectBins==1))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of tones drawn from the gaussian pdfs',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/GaussMeanToneProbHighBadSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,0,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,0,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,1,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,1,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,2,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,2,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,3,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins==2,3,:],axis=0)/sum(subjectBins==2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of tones drawn from the gaussian pdfs',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/GaussMeanToneProbHighMediocreSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,0,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,0,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,1,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,1,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,2,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,2,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,3,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArray[AllTonesGaussianAccuracy>=70][subjectBins>2,3,:],axis=0)/sum(subjectBins>2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of tones drawn from the gaussian pdfs',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/GaussMeanToneProbHighGoodSubj')


In [ ]:
"""
Accuracy vs % of data explained using strategies for all subjects in the long context case biased towards low
"""

def computeAggregateBiasedLong():
    AllTonesGaussianMeanExplainedLc = np.copy(AllTonesGaussianAccuracyLc)
    TwoDistOppSidebandSignalMeanExplainedLc = 100-TwoDistOppSidebandMeanExplainedLc
    AllTonesGaussianMeanExplainedHc = np.copy(AllTonesGaussianAccuracyHc)
    TwoDistOppSidebandSignalMeanExplainedHc = 100-TwoDistOppSidebandMeanExplainedHc
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot([0,1,2,3],[noDistractorAccuracyLc*100,
                       oneDistractorAccuracyLc*100,
                       twoDistractorsAccuracyLc*100,
                       threeDistractorsAccuracyLc*100],'k.')
    ax.set_ylim(0,100)
    ax.set_xlim(-0.5,4.5)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Number of Distractors',fontsize=15)
    plt.ylabel('Accuracy',fontsize=15)
    plt.tight_layout()
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(AllTonesGaussianAccuracyLc, AllTonesGaussianMeanExplainedLc, 'b.',markersize=18)
    ax.plot(AllTonesGaussianAccuracyHc, AllTonesGaussianMeanExplainedHc, 'skyblue',
            marker='.',markersize=18,linestyle = 'None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks(fontsize=15)
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/AllTonesGaussAccuracyMeanStrategy')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(AllTonesGaussianAccuracyLc, AllTonesGaussianMeanExplainedLc, 'r.',markersize=18)
    ax.plot(AllTonesGaussianAccuracyHc, AllTonesGaussianMeanExplainedHc, 'lightsalmon',
            marker='.',markersize=18,linestyle = 'None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks(fontsize=15)
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/AllTonesGaussAccuracySignalMeanStrategy')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(AllTonesGaussianAccuracyLc, AllTonesGaussianMeanExplainedLc, 'g.',markersize=18)
    ax.plot(AllTonesGaussianAccuracyHc, AllTonesGaussianMeanExplainedHc, 'limegreen',
            marker='.',markersize=18,linestyle = 'None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/AllTonesGaussAccuracyVotingStrategy')    

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistGaussianAccuracyLc, OneDistGaussianMeanExplainedLc, 'b.', markersize=18,)
    ax.plot(OneDistGaussianAccuracyHc, OneDistGaussianMeanExplainedHc, 'skyblue',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistGaussAccuracyMeanStrategy')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistGaussianAccuracyLc, OneDistGaussianSignalMeanExplainedLc, 'r.', markersize=18)
    ax.plot(OneDistGaussianAccuracyHc, OneDistGaussianSignalMeanExplainedHc, 'lightsalmon',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistGaussAccuracySignalMeanStrategy')  
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistGaussianAccuracyLc, OneDistGaussianSignalMeanExplainedLc, 'g.', markersize=18)
    ax.plot(OneDistGaussianAccuracyHc, OneDistGaussianSignalMeanExplainedHc, 'limegreen',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks(fontsize=15)
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistGaussAccuracyVotingStrategy')       

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistOppSidebandLHAccuracyLc, OneDistOppSidebandLHMeanExplainedLc, 'b.', markersize=18)
    ax.plot(OneDistOppSidebandLHAccuracyHc, OneDistOppSidebandLHMeanExplainedHc, 'skyblue',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistOppSidebandLHAccuracyMeanStrategy')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistOppSidebandLHAccuracyLc, OneDistOppSidebandLHSignalMeanExplainedLc, 'r.', markersize=18)
    ax.plot(OneDistOppSidebandLHAccuracyHc, OneDistOppSidebandLHSignalMeanExplainedHc, 'lightsalmon',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistOppSidebandLHAccuracySignalMeanStrategy') 
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistOppSidebandLHAccuracyLc, OneDistOppSidebandLHSignalMeanExplainedLc, 'g.', markersize=18)
    ax.plot(OneDistOppSidebandLHAccuracyHc, OneDistOppSidebandLHSignalMeanExplainedHc, 'limegreen',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks(fontsize=15)
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistOppSidebandLHAccuracyVotingStrategy')      

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistOppSidebandHLAccuracyLc, OneDistOppSidebandHLMeanExplainedLc, 'b.', markersize=18)
    ax.plot(OneDistOppSidebandHLAccuracyHc, OneDistOppSidebandHLMeanExplainedHc, 'skyblue',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistOppSidebandHLAccuracyMeanStrategy')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistOppSidebandHLAccuracyLc, OneDistOppSidebandHLSignalMeanExplainedLc, 'r.', markersize=18)
    ax.plot(OneDistOppSidebandHLAccuracyHc, OneDistOppSidebandHLSignalMeanExplainedHc, 'lightsalmon',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistOppSidebandHLAccuracySignalMeanStrategy')  
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(OneDistOppSidebandHLAccuracyLc, OneDistOppSidebandHLSignalMeanExplainedLc, 'g.', markersize=18)
    ax.plot(OneDistOppSidebandHLAccuracyHc, OneDistOppSidebandHLSignalMeanExplainedHc, 'limegreen',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.yticks([])
    plt.xticks(fontsize=15)
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/OneDistOppSidebandHLAccuracyVotingStrategy')      

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(TwoDistOppSidebandAccuracyLc, TwoDistOppSidebandMeanExplainedLc, 'b.', markersize=18)
    ax.plot(TwoDistOppSidebandAccuracyHc, TwoDistOppSidebandMeanExplainedHc, 'skyblue', 
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/TwoDistOppSidebandAccuracyMeanStrategy')
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(TwoDistOppSidebandAccuracyLc, TwoDistOppSidebandSignalMeanExplainedLc, 'r.', markersize=18)
    ax.plot(TwoDistOppSidebandAccuracyHc, TwoDistOppSidebandSignalMeanExplainedHc, 'lightsalmon', 
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/TwoDistOppSidebandAccuracySignalMeanStrategy')   
    
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    ax.plot(TwoDistOppSidebandAccuracyLc, TwoDistOppSidebandMeanExplainedLc, 'g.', markersize=18)
    ax.plot(TwoDistOppSidebandAccuracyHc, TwoDistOppSidebandMeanExplainedHc, 'limegreen',
            marker='.', markersize=18, linestyle='None')
    ax.set_ylim(0,100)
    ax.set_xlim(0,100)
    plt.xticks(fontsize=15)
    plt.yticks([])
    plt.tight_layout()
    plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/TwoDistOppSidebandAccuracyVotingStrategy')    

    """
    Plotting average p(B_H|mean tone) and average p(B_H|mean of signal tones)
    """
    _, bin_edges = np.histogram((OneDistOppSidebandHLAccuracyLc+OneDistOppSidebandLHAccuracyLc)/2,bins=3)
    subjectBins = np.digitize((OneDistOppSidebandHLAccuracyLc+OneDistOppSidebandLHAccuracyLc)/2, bin_edges)
    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==1,0,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==1,0,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==1,1,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==1,1,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==1,2,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==1,2,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==1,3,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==1,3,:],axis=0)/sum(subjectBins==1))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of all 3 tones',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/AllMeanToneProbHighBadSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==2,0,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==2,0,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==2,1,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==2,1,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==2,2,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==2,2,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins==2,3,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins==2,3,:],axis=0)/sum(subjectBins==2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of all 3 tones',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/AllMeanToneProbHighMediocreSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins>2,0,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins>2,0,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins>2,1,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins>2,1,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins>2,2,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins>2,2,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(allMeanTone_behvSubjArrayLc[subjectBins>2,3,:],axis=0), 
             yerr=np.std(allMeanTone_behvSubjArrayLc[subjectBins>2,3,:],axis=0)/sum(subjectBins>2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of all 3 tones',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/AllMeanToneProbHighGoodSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==1,0,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==1,0,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==1,1,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==1,1,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==1,2,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==1,2,:],axis=0)/sum(subjectBins==1))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==1,3,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==1,3,:],axis=0)/sum(subjectBins==1))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of tones drawn from the gaussian pdfs',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/GaussMeanToneProbHighBadSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==2,0,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==2,0,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==2,1,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==2,1,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==2,2,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==2,2,:],axis=0)/sum(subjectBins==2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins==2,3,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins==2,3,:],axis=0)/sum(subjectBins==2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of tones drawn from the gaussian pdfs',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/GaussMeanToneProbHighMediocreSubj')

    fig = plt.figure()  # create a figure object
    ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins>2,0,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins>2,0,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins>2,1,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins>2,1,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins>2,2,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins>2,2,:],axis=0)/sum(subjectBins>2))
    plt.errorbar(logFreqBins, np.mean(GaussMeanTone_behvSubjArrayLc[subjectBins>2,3,:],axis=0), 
             yerr=np.std(GaussMeanTone_behvSubjArrayLc[subjectBins>2,3,:],axis=0)/sum(subjectBins>2))
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel('Mean of tones drawn from the gaussian pdfs',fontsize=15)
    plt.ylabel('p(B_H|mean(T))',fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/biasedLow_rawDataAnalysis/GaussMeanToneProbHighGoodSubj')


In [ ]:
"""
Executing both aggregate functions to generate plots
"""
computeAggregate()
computeAggregateBiasedLong()

In [ ]:
"""
Testing for subject bias in the long context case
"""
labels = np.arange(20)
iSubjNoandLow = np.delete(np.arange(33),[0,5,11,12,19,26,30,32])
iSubjNoandLowandHigh = np.delete(np.arange(33),[0,1,2,5,9,11,12,19,23,26,28,30,32])
iSubjLowandHigh = np.delete(np.arange(25),[0,1,7,18,22])
biasNoContext = np.array([0.2,0,0.33,-0.12,0.23,-0.27,-0.1,0.43,0.33,0.21,0.16,
                          0.52,0.2,0.09,0.21,0.27,0.31,0.33,0.095,0.02])
biasLongContextLow = np.array([-0.3,-1.65,-0.49,-0.71,-0.24,-0.35,-0.42,0.05,-0.6,-0.81,
                              0.026,-1.09,-0.83,-0.81,-0.63,-0.36,-0.39,-0.32,-0.54,-0.58])
biasLongContextHigh = np.array([0.82,1,0.88,0.35,1.05,0.27,0.56,1.12,0.39,0.74,0.61,
                                1.69,0.46,0.27,1.34,0.82,0.77,0.3,0.73,1.34])

x = np.arange(len(labels))  # the label locations
width = 0.4  # the width of the bars

fig, ax = plt.subplots()
ax.bar(x - width/2, OneDistOppSidebandHLAccuracy[iSubjNoandLowandHigh]/100-
       OneDistOppSidebandLHAccuracy[iSubjNoandLowandHigh]/100, width, color='k')
ax.bar(x + width/2, OneDistOppSidebandHLAccuracyLc[iSubjLowandHigh]/100-
       OneDistOppSidebandLHAccuracyLc[iSubjLowandHigh]/100, width, color=[0.8,0.8,0.8])
ax.bar(x + width/2, OneDistOppSidebandHLAccuracyHc/100-
       OneDistOppSidebandLHAccuracyHc/100, width, color=[0.5,0.5,0.5])
plt.xticks(fontsize=15,ticks=np.arange(0,len(labels),2),labels=np.arange(0,len(labels),2))
plt.yticks(fontsize=15)
plt.xlabel('Subjects',fontsize=15)
plt.ylabel('Difference in accuracies',fontsize=15)
plt.tight_layout()
plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/barplot=Difference in accuracies given one distractor from sidebands')

fig, ax = plt.subplots()
ax.bar(x - width/2, allGaussLowAccuracy[iSubjNoandLowandHigh]-
       allGaussHighAccuracy[iSubjNoandLowandHigh], width, color='k')
ax.bar(x + width/2, allGaussLowAccuracyLc[iSubjLowandHigh]-
       allGaussHighAccuracyLc[iSubjLowandHigh], width, color=[0.8,0.8,0.8])
ax.bar(x + width/2, allGaussLowAccuracyHc-
       allGaussHighAccuracyHc, width, color=[0.5,0.5,0.5])
plt.xticks(fontsize=15,ticks=np.arange(0,len(labels),2),labels=np.arange(0,len(labels),2))
plt.yticks(fontsize=15)
plt.xlabel('Subjects',fontsize=15)
plt.ylabel('Difference in accuracies',fontsize=15)
plt.tight_layout()
plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/barplot=Difference in accuracies given all gauss tones')

fig, ax = plt.subplots()
ax.bar(x - width/2, distGaussHighAccuracy[iSubjNoandLowandHigh]-
       distGaussLowAccuracy[iSubjNoandLowandHigh], width, color='k')
ax.bar(x + width/2, distGaussHighAccuracyLc[iSubjLowandHigh]-
       distGaussLowAccuracyLc[iSubjLowandHigh], width, color=[0.8,0.8,0.8])
ax.bar(x + width/2, distGaussHighAccuracyHc-
       distGaussLowAccuracyHc, width, color=[0.5,0.5,0.5])
plt.xticks(fontsize=15,ticks=np.arange(0,len(labels),2),labels=np.arange(0,len(labels),2))
plt.yticks(fontsize=15)
plt.xlabel('Subjects',fontsize=15)
plt.ylabel('Difference in accuracies',fontsize=15)
plt.tight_layout()
plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/barplot=Difference in accuracies given one distractor from gaussian region')

fig, ax = plt.subplots()
ax.plot(np.abs(allGaussLowAccuracyLc[iSubjLowandHigh]-allGaussHighAccuracyLc[iSubjLowandHigh]-
        (allGaussLowAccuracy[iSubjNoandLowandHigh]-allGaussHighAccuracy[iSubjNoandLowandHigh])),
        -biasLongContextLow+biasNoContext,'k.')
ax.plot(np.abs(allGaussLowAccuracyHc-allGaussHighAccuracyHc-
        (allGaussLowAccuracy[iSubjNoandLowandHigh]-allGaussHighAccuracy[iSubjNoandLowandHigh])),
        biasLongContextHigh-biasNoContext,'k.')
plt.xlabel('Difference in accuracies across contexts',fontsize=15)
plt.ylabel('Difference in w0',fontsize=15)
plt.tight_layout()
#plt.savefig('figures/FromProlific/biasedLongContext_rawDataAnalysis/barplot=Difference in accuracies given one distractor from gaussian region')



In [ ]:
def checkBiasUsingCentralTones(trial_tones, trial_behaviour, corrans):
    twoCentralTones = np.zeros((len(trial_behaviour),12))
    
    n_tones = len(trial_tones[0])
    print(high_dist[0]+1.8*high_dist[1], 
          low_dist[0]-1.8*low_dist[1], 
          log_freq_seq_mid)
    
    iTwoCentralTones = 0;
    
    for i_trial in range(len(trial_tones)):
        if ((int(log_freq_seq_mid-0.075<trial_tones[i_trial][0]<log_freq_seq_mid+0.075) +
             int(log_freq_seq_mid-0.075<trial_tones[i_trial][1]<log_freq_seq_mid+0.075) +
             int(log_freq_seq_mid-0.075<trial_tones[i_trial][2]<log_freq_seq_mid+0.075))==2):
            
            twoCentralTones[iTwoCentralTones,:3] = trial_tones[i_trial,:]
            twoCentralTones[iTwoCentralTones,3] = trial_behaviour[i_trial]
            twoCentralTones[iTwoCentralTones,4] = corrans[i_trial]
            iTwoCentralTones += 1       
            print(trial_tones[i_trial])
            
    return twoCentralTones[:iTwoCentralTones,:]

for iSubj in range(len(trial_behaviour_expt)):
    twoCentralTones = checkBiasUsingCentralTones(np.log10(trial_tones_expt[iSubj]),
                                                trial_behaviour_expt[iSubj], 
                                                corrans_expt[iSubj])
    
    unique_tones = np.unique(twoCentralTones[:,:3])

    tone1_prob_behaviour = np.zeros((len(unique_tones)))
    tone2_prob_behaviour = np.zeros((len(unique_tones)))
    tone3_prob_behaviour = np.zeros((len(unique_tones)))

    for i_tone in range(len(unique_tones)):
        tone1_prob_behaviour[i_tone] = np.mean(twoCentralTones[twoCentralTones[:,0]\
                                                           ==unique_tones[i_tone],3])
        tone2_prob_behaviour[i_tone] = np.mean(twoCentralTones[twoCentralTones[:,1]\
                                                           ==unique_tones[i_tone],3])
        tone3_prob_behaviour[i_tone] = np.mean(twoCentralTones[twoCentralTones[:,2]\
                                                           ==unique_tones[i_tone],3])
    plt.plot(unique_tones,tone1_prob_behaviour)
    plt.plot(unique_tones,tone2_prob_behaviour)
    plt.plot(unique_tones,tone3_prob_behaviour)
    

In [ ]:
"""
Code for initial plots showing how accuracy varies with frequency of distractors.
Note that the code for fn. 'accuracies' was changed to create the following plots. 
"""

fig = plt.figure()  # create a figure object
ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure
ax.errorbar(x=[0,1,2,3],y=[np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,4]*100),
         np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,0]*100),
         np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,3]*100),
         np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,7]*100)],
        yerr=[np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,4]*100)/np.sqrt(30),
         np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,0]*100)/np.sqrt(30),
         np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,3]*100)/np.sqrt(30),
         np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,7]*100)/np.sqrt(30)],
            color='k')
ax.errorbar(x=[0,1,2,3],y=[np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,6]*100),
         np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,2]*100),
         np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,1]*100),
         np.mean(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,5]*100)],
        yerr=[np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,6]*100)/np.sqrt(30),
         np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,2]*100)/np.sqrt(30),
         np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,1]*100)/np.sqrt(30),
         np.std(accuracyAcrossFreq[(accuracyAcrossFreq[:,0]+accuracyAcrossFreq[:,1])>=1.4,5]*100)/np.sqrt(30)],
           color='r')
ax.set_ylim(0,100)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Frequency',fontsize=15)
plt.ylabel('Accuracy',fontsize=15)
plt.tight_layout()
#plt.savefig('figures/FromProlific/rawDataAnalysis/AccuracyVariationWithFrequency')


In [ ]:
def sample_expt(params, pl_orig):
    training_trials, dist_chosen, tone_kind, low_dist, high_dist = task(freq_seq = freqSeq, 
                                                                        n_trials = 20000, n_tones = 3, 
                                                                        p_back=0.3, p_low=pl_orig,
                                                                        lm=2.55,hm=2.85,s=0.1)

    all_trial_tones, all_trial_behaviour = generate_behaviour(training_trials, reps=1, n_tones=3, 
                                                              prob_back=params[4], prob_low=params[5], 
                                                              log_prior_params=params[:3], 
                                                              sigma_sensory=params[3])
    return all_trial_tones, all_trial_behaviour, dist_chosen

def sample_exptVoting(params,pl_orig):
    training_trials, dist_chosen, tone_kind, low_dist, high_dist = task(freq_seq = freqSeq, 
                                                                        n_trials = 20000, n_tones = 3, 
                                                                        p_back=0.3, p_low=pl_orig,
                                                                        lm=2.55,hm=2.85,s=0.1)

    all_trial_tones, all_trial_behaviour = generate_behaviourVoting(training_trials, reps=1, n_tones=3, 
                                                                    decision_boundary=params[0],
                                                                    sigma_sensory=params[1])
    return all_trial_tones, all_trial_behaviour, dist_chosen

def posteriorAgainstPercept(expt_Params):
    [_,_,mle_LikelihoodLatentTonegivenHigh,
    mle_LikelihoodLatentTonegivenLow,_,mle_posterior] = posterior_array(freq_input=log_freq_perceptFinelySampled,
                                                                        n_tones=3,
                                                                        p_back=expt_Params[4], 
                                                                        p_low=expt_Params[5],
                                                                        log_prior=expt_Params[:3]) 
    
    
    mle_LikelihoodPerceptgivenHigh = np.zeros((len(log_freq_perceptFinelySampled),
                                               len(log_freq_perceptFinelySampled),len(log_freq_perceptFinelySampled)))
    mle_LikelihoodPerceptgivenLow = np.zeros((len(log_freq_perceptFinelySampled),
                                              len(log_freq_perceptFinelySampled),len(log_freq_perceptFinelySampled)))

    for itrue1 in range(len(log_freq_perceptFinelySampled)):
        for itrue2 in range(len(log_freq_perceptFinelySampled)):
            for itrue3 in range(len(log_freq_perceptFinelySampled)):
                mle_probPerceptgivenLatentTones = Tones3dgrid([log_freq_perceptFinelySampled[itrue1],
                                                               log_freq_perceptFinelySampled[itrue2],
                                                               log_freq_perceptFinelySampled[itrue3]],
                                                               freq_seq = log_freq_perceptFinelySampled,
                                                               sigma=expt_Params[3])
                mle_LikelihoodPerceptgivenHigh \
                += mle_probPerceptgivenLatentTones * mle_LikelihoodLatentTonegivenHigh[itrue1,itrue2,itrue3]
                mle_LikelihoodPerceptgivenLow \
                += mle_probPerceptgivenLatentTones * mle_LikelihoodLatentTonegivenLow[itrue1,itrue2,itrue3]
    mle_probHighgivenPercept = mle_LikelihoodPerceptgivenHigh*(1-expt_Params[5])/\
    (mle_LikelihoodPerceptgivenHigh*(1-expt_Params[5]) + mle_LikelihoodPerceptgivenLow*expt_Params[5])
    return mle_probHighgivenPercept


In [ ]:
"""
For three participants, overall high, medium and low accuracy show the following plots of how well the Bayesian
model, voting model and null Bayesian model captures features in data.
"""

BayesRsquared = np.zeros((len(mleParams_expt),))
NullRsquared = np.zeros((len(lesionParams_expt),))
VotingRsquared = np.zeros((len(votingParams_expt),))

for iSubj in [3]:#range(len(mleParams_expt)):
    
    plt.figure()    
    """
    unique_tones, exptBhv_mean, exptBhv_std = plotting_pBHgivenT(subjectBehaviour = trial_behaviour_expt[iSubj], 
                                                                 subjectTones = trial_tones_expt[iSubj], 
                                                                 contextIndex = range(len(trial_behaviour_expt[iSubj])))
    plt.errorbar(np.log10(unique_tones), exptBhv_mean, yerr=exptBhv_std/np.sqrt(3), color='black') 

    mleTrialTones, mleTrialBehaviour, mleCorrNum = sample_expt(params=mleParams_expt[iSubj],
                                                               pl_orig=0.5)

    mleUnique_tones, mleBhv_mean, mleBhv_std = plotting_pBHgivenT(subjectBehaviour = mleTrialBehaviour, 
                                                               subjectTones = mleTrialTones, 
                                                               contextIndex = range(len(mleTrialBehaviour)))
    plt.errorbar(np.log10(mleUnique_tones), mleBhv_mean, yerr=mleBhv_std/np.sqrt(3), color='red')
    
    mleTrialTones, mleTrialBehaviour, mleCorrNum = sample_expt(params=[2.54,2.85,0.016,0.16,0.051,0.47],
                                                               pl_orig=0.5)
    
    mleUnique_tones, mleBhv_mean, mleBhv_std = plotting_pBHgivenT(subjectBehaviour = mleTrialBehaviour, 
                                                               subjectTones = mleTrialTones, 
                                                               contextIndex = range(len(mleTrialBehaviour)))
    plt.errorbar(np.log10(mleUnique_tones), mleBhv_mean, yerr=mleBhv_std/np.sqrt(3), color='blue')
    """
    #lesionTrialTones, lesionTrialBehaviour, lesionCorrNum = sample_expt(params=lesionParams_expt[iSubj],
    #                                                                    pl_orig=0.5)

    #unique_tones, lesionBhv_mean, lesionBhv_std = plotting_pBHgivenT(subjectBehaviour = lesionTrialBehaviour, 
    #                                                                 subjectTones = lesionTrialTones, 
    #                                                                 contextIndex = range(len(lesionTrialBehaviour)))
    #plt.errorbar(np.log10(unique_tones), lesionBhv_mean, yerr=lesionBhv_std/np.sqrt(3), color='orange')

    #votingTrialTones, votingTrialBehaviour, votingCorrNum = sample_exptVoting(params=votingParams_expt[iSubj],
    #                                                                          pl_orig=0.5)

    #unique_tones, votingBhv_mean, votingBhv_std = plotting_pBHgivenT(subjectBehaviour = votingTrialBehaviour, 
    #                                                                 subjectTones = votingTrialTones, 
    #                                                                 contextIndex = range(len(votingTrialBehaviour)))
    #plt.errorbar(np.log10(unique_tones), votingBhv_mean, yerr=votingBhv_std/np.sqrt(3), color='dodgerblue') 
    
    plt.xlabel('Tones',fontsize=15)
    plt.ylabel('p(B_H|T)',fontsize=15)
    plt.xticks(ticks=np.arange(1.9,3.6,0.3), labels=np.around(10**np.arange(1.9,3.6,0.3),2),fontsize=13)
    plt.yticks(fontsize=15)
    plt.tight_layout()
    #plt.savefig('figures/FromProlific/rawDataAnalysis/experimenter=2099_cleaner_modelfit.png')
    
    #BayesRsquared[iSubj] = psuedoRsquared(fitY=mleBhv_mean,
    #                                        exptY=exptBhv_mean,
    #                                        exptSem=exptBhv_std/np.sqrt(3))
    #NullRsquared[iSubj] = psuedoRsquared(fitY=lesionBhv_mean,
    #                                     exptY=exptBhv_mean,
    #                                     exptSem=exptBhv_std/np.sqrt(3))
    #VotingRsquared[iSubj] = psuedoRsquared(fitY=votingBhv_mean,
    #                                     exptY=exptBhv_mean,
    #                                     exptSem=exptBhv_std/np.sqrt(3))
    log_freq_perceptFinelySampled=np.arange(0.6,4.7,0.02)
    """
    Comparing posteriors obtained using original mle parameters, and those derived from min and max 
    (sigma_s, sigma_ss) ranges.
    """
    origMLE_probHighgivenPercept = posteriorAgainstPercept([2.54,2.85,0.016,0.16,0.05,0.47])
    
    tone1_prob_behaviour = np.zeros((len(log_freq_perceptFinelySampled)))
    tone2_prob_behaviour = np.zeros((len(log_freq_perceptFinelySampled)))
    tone3_prob_behaviour = np.zeros((len(log_freq_perceptFinelySampled)))

    for i_tone in range(len(log_freq_perceptFinelySampled)):
        tone1_prob_behaviour[i_tone] = np.mean(origMLE_probHighgivenPercept[i_tone,:,:])
        tone2_prob_behaviour[i_tone] = np.mean(origMLE_probHighgivenPercept[:,i_tone,:])
        tone3_prob_behaviour[i_tone] = np.mean(origMLE_probHighgivenPercept[:,:,i_tone])
    
    plt.figure()
    plt.plot(log_freq_perceptFinelySampled, (tone1_prob_behaviour+tone2_prob_behaviour+tone3_prob_behaviour)/3, 
             color='k')
    plt.show()
    
    minMLE_probHighgivenPercept = posteriorAgainstPercept(mleParams_expt[iSubj,:])
    
    tone1_prob_behaviour = np.zeros((len(log_freq_perceptFinelySampled)))
    tone2_prob_behaviour = np.zeros((len(log_freq_perceptFinelySampled)))
    tone3_prob_behaviour = np.zeros((len(log_freq_perceptFinelySampled)))

    for i_tone in range(len(log_freq_perceptFinelySampled)):
        tone1_prob_behaviour[i_tone] = np.mean(minMLE_probHighgivenPercept[i_tone,:,:])
        tone2_prob_behaviour[i_tone] = np.mean(minMLE_probHighgivenPercept[:,i_tone,:])
        tone3_prob_behaviour[i_tone] = np.mean(minMLE_probHighgivenPercept[:,:,i_tone])
        
    plt.plot(log_freq_perceptFinelySampled, (tone1_prob_behaviour+tone2_prob_behaviour+tone3_prob_behaviour)/3, 
             'k--')
    plt.xlabel('Tones',fontsize=15)
    plt.ylabel('p(B_H|T)',fontsize=15)
    plt.xticks(ticks=np.arange(0.6,4.7,1), labels=np.around(10**np.arange(0.6,4.7,1),2),fontsize=15)
    plt.yticks(fontsize=15)
    plt.tight_layout()    
        
    #plt.savefig('figures/FromProlific/rawDataAnalysis/experimenter=e453_posteriorUsingPercepts.png')
    
    #lowProbDist, highProbDist = visualizeProbDistributions(sample_x=log_freq_percept,
    #                                                       log_freq_low=mleParams_expt[iSubj,[0,2]],
    #                                                       log_freq_high=mleParams_expt[iSubj,[1,2]])
    #plt.figure()
    #plt.plot(log_freq_percept,(1-mleParams_expt[iSubj,4])*mleParams_expt[iSubj,5]*lowProbDist/(1-mleParams_expt[iSubj,5]),'k')
    #plt.plot(log_freq_percept,(1-mleParams_expt[iSubj,4])*(1-mleParams_expt[iSubj,5])*highProbDist/mleParams_expt[iSubj,5],'k')
    #plt.plot(log_freq_percept,mleParams_expt[iSubj,4]*np.ones(len(log_freq_percept))/len(log_freq_percept),'skyblue')
    #plt.xlabel('Tones',fontsize=15)
    #plt.ylabel('Probability',fontsize=15)
    #plt.xticks(ticks=np.arange(0.9,4.7,0.8), labels=np.around(10**np.arange(0.9,4.7,0.8),2),fontsize=14)
    #plt.yticks(fontsize=15)
    #plt.tight_layout()    
    #plt.savefig('figures/FromProlific/rawDataAnalysis/experimenter=1c3f_probabilityDistributions.png')
    
#print('Difference between Bayes and Null',scipy.stats.ks_2samp(BayesRsquared, NullRsquared))
#print('Difference between Bayes and voting',scipy.stats.ks_2samp(BayesRsquared, VotingRsquared))

In [ ]:
subjects = ['735a','1304','6b7f','1c3f','1604','1396','b4c7','a12e','c653','4b7f','fc3a','0d04',
           '2099','592d','a45d','e453','214b','b7cc','188a','e12e']

for ii in range(len(trial_behaviour_exptHc)):
    plt.figure()
    """
    Comparing to low context experiment
    """
    idxHigh = np.arange(len(trial_behaviour_exptLc[iSubjLowandHigh[ii]][80:]))[corrans_exptLc[iSubjLowandHigh[ii]][80:]==1]
    idxLow = np.arange(len(trial_behaviour_exptLc[iSubjLowandHigh[ii]][80:]))[corrans_exptLc[iSubjLowandHigh[ii]][80:]==0]
    idxToKeep = np.concatenate((idxHigh, np.random.choice(idxLow,size=len(idxHigh),replace=False)))
    print(len(idxToKeep))
    
    unique_tones, bhvLow_mean, bhvLow_std = plotting_pBHgivenT(subjectBehaviour = trial_behaviour_exptLc[iSubjLowandHigh[ii]][80:], 
                                                               subjectTones = trial_tones_exptLc[iSubjLowandHigh[ii]][80:], 
                                                               contextIndex = idxToKeep)
    plt.errorbar(np.log10(unique_tones), bhvLow_mean, yerr=bhvLow_std/np.sqrt(3), color='blue') 
    
    """
    Comparing to high context experiment
    """
    idxHigh = np.arange(len(trial_behaviour_exptHc[ii][80:]))[corrans_exptHc[ii][80:]==1]
    idxLow = np.arange(len(trial_behaviour_exptHc[ii][80:]))[corrans_exptHc[ii][80:]==0]
    idxToKeep = np.concatenate((idxLow, np.random.choice(idxHigh,size=len(idxLow),replace=False)))
    print(len(idxToKeep))
    
    unique_tones, bhvHigh_mean, bhvHigh_std = plotting_pBHgivenT(subjectBehaviour = trial_behaviour_exptHc[ii][80:], 
                                                                 subjectTones = trial_tones_exptHc[ii][80:], 
                                                                 contextIndex = idxToKeep)
    plt.errorbar(np.log10(unique_tones), bhvHigh_mean, yerr=bhvHigh_std/np.sqrt(3), color='green') 
    
    """
    Comparing to no context experiment
    """
    unique_tones, bhvNo_mean, bhvNo_std = plotting_pBHgivenT(subjectBehaviour = trial_behaviour_expt[iSubjNoandLowandHigh[ii]], 
                                                             subjectTones = trial_tones_expt[iSubjNoandLowandHigh[ii]], 
                                                             contextIndex = range(len(trial_behaviour_expt
                                                                                      [iSubjNoandLowandHigh[ii]])))
    plt.errorbar(np.log10(unique_tones), bhvNo_mean, yerr=bhvNo_std/np.sqrt(3), color='red') 
    
    """
    Comparing to low context simulations
    
    simLowTones, simLowBehaviour, simLowCorrNum = sample_expt(params=mleParams_exptLc[iSubjLowandHigh[ii]],
                                                              pl_orig=0.7)

    idxSimHigh = np.arange(len(simLowBehaviour))[simLowCorrNum==1]
    idxSimLow = np.arange(len(simLowBehaviour))[simLowCorrNum==0]
    idxSimToKeep = np.concatenate((idxSimHigh, np.random.choice(idxSimLow,size=len(idxSimHigh),replace=False)))
    
    unique_tones, simBhvLow_mean, simBhvLow_std = plotting_pBHgivenT(subjectBehaviour = simLowBehaviour, 
                                                                    subjectTones = simLowTones, 
                                                                    contextIndex = idxSimToKeep)
    plt.errorbar(np.log10(unique_tones), simBhvLow_mean, yerr=simBhvLow_std/np.sqrt(3), color='grey')
    
    
    Comparing to no context simulations
    
    simNoTones, simNoBehaviour, simNoCorrNum  = sample_expt(params=mleParams_expt[iSubjNoandLowandHigh[ii]],
                                                            pl_orig=0.5)

    unique_tones, simBhvNo_mean, simBhvNo_std = plotting_pBHgivenT(subjectBehaviour = simNoBehaviour, 
                                                                   subjectTones = simNoTones, 
                                                                   contextIndex = range(len(simNoBehaviour)))
    plt.errorbar(np.log10(unique_tones), simBhvNo_mean, yerr=simBhvNo_std/np.sqrt(3), color='black')
    """
    plt.xlabel('log10(Tones)',fontsize=15)
    plt.ylabel('p(B_H|T)',fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.tight_layout()
    plt.savefig(f'figures/FromProlific/biasedLongContext_rawDataAnalysis/{subjects[ii]}_pBHgivenT_forNoandLongContext')

In [ ]:
for jj in range(len(iSubjNoandLowandHigh)):
    accuracyOverTime = np.zeros(len(trial_behaviour_expt[iSubjNoandLowandHigh[jj]]))
    plt.figure()
    for tr in range(20,len(trial_behaviour_expt[iSubjNoandLowandHigh[jj]])):
        accuracyOverTime[tr] = np.sum(trial_behaviour_expt[iSubjNoandLowandHigh[jj]][tr-20:tr]
                           ==corrans_expt[iSubjNoandLowandHigh[jj]][tr-20:tr])/20
        #plt.plot(np.cumsum(trial_behaviour_exptLc[iSubjLowandHigh[jj]][tr-10:tr]
        #                   ==corrans_exptLc[iSubjLowandHigh[jj]][tr-10:tr])/10,color='blue')
        #plt.plot(np.cumsum(trial_behaviour_exptHc[jj][tr-10:tr]
        #                   ==corrans_exptHc[jj][tr-10:tr])/10,color='green')
    plt.plot(accuracyOverTime)    
    plt.xlim([-10,600])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
        #plt.savefig(f'figures/FromProlific/biasedLongContext_rawDataAnalysis/learningRate_Subj{jj}')
    